<!-- VISIBLE TITLE BLOCK -->
<h1 style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif;
            background-color: #f9fbfbff;
            color: #38545f;
            border-top: 6px solid #ec2625;
            padding: 20px 15px 15px 15px;">
    <span style="font-size: 32px; font-weight: bold;">Ho Chi Minh City Temperature Forecasting</span>
    <br>
    <span style="font-size: 20px; color: #6c8794;">Group 2 - Analysis & Implementation</span>
</h1>

- This notebook presents a comprehensive, methodologically rigorous approach to forecasting the daily average temperature in Ho Chi Minh City for the next five days
- The process documented here follows the project brief from data sourcing and deep exploratory analysis to the final, interpretable champion model
- Every decision is justified with data-driven evidence, prioritizing sound methodology to ensure a robust and reliable outcome.

<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="setup" style="display: none;">0. Setup & Global Configuration</h1>
<!-- VISIBLE H1 -->
<h1 id="setup-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">0. Setup & Global Configuration</h1>

In [1]:
# --- Core Libraries ---
import pandas as pd
import numpy as np
import warnings
import os
import joblib
from typing import List, Dict, Tuple, Any

# --- Machine Learning ---
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import MultiTaskLassoCV

# --- Visualization & Utilities ---
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from rich.console import Console
from rich.table import Table

# --- Custom Project Modules ---
import utils.custom_plotly_style_te as cpte
import utils.eda_visualizations as eda
import utils.analysis_helpers as ah
import utils.evaluations as eval
import utils.diagnostics as diag


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="config" style="display: none;">0.1. Centralized Pipeline Configuration</h2>
<!-- VISIBLE H2 -->
<h2 id="config-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">0.1. Centralized Pipeline Configuration</h2>

In [ ]:
class PipelineConfig:
    """
    A centralized configuration class for managing paths, parameters, and flags
    throughout the notebook. Reproducibility? Yea. Quick and easy key settings
    modifications? Also yea:)
    """

    # --- Path & File Configuration ---
    INPUT_DATA_PATH: str = "data/weather_hcm_daily.csv"
    # ARTIFACTS_DIR: str = "champion_model_artifacts"
    UTILS_DIR: str = "utils"

    # --- Model Training Control ---
    # Set to True to retrain and save models.
    # Set to False to load pre-trained models for a faster run.
    TRAIN_NEW_MODELS: bool = False

    # --- General Model Parameters ---
    GLOBAL_RANDOM_SEED: int = 105
    HORIZONS: List[int] = [1, 2, 3, 4, 5] # Predict from t+1 to t+5
    TARGET_VARIABLE: str = "temp"

    # --- Data Split Parameters ---
    # Using an 85/15 split for a substantial test set. We use TimeSeriesSplit CV so can do without three-way holdout
    TEST_SIZE: float = 0.15


# Instantiate the config class and create necessary directories
config = PipelineConfig()
# os.makedirs(config.ARTIFACTS_DIR, exist_ok=True)
os.makedirs(config.UTILS_DIR, exist_ok=True)

# Instantiate console for rich printing
console = Console()


In [3]:
# --- Global Stylistic & Environment Configuration ---

# Suppress warnings for a cleaner notebook presentation.
warnings.filterwarnings("ignore")

pio.templates["te"] = cpte.te_style_template
pio.templates.default = "te"
pd.options.plotting.backend = "plotly"


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="sourcing" style="display: none;">Q1. Data Sourcing</h1>
<!-- VISIBLE H1 -->
<h1 id="sourcing-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">Q1. Data Sourcing</h1>

This section details the methodology used to collect the historical weather data for Ho Chi Minh City, which forms the foundation of this forecasting project.

*   **Data Source:** [Visual Crossing Weather History API](https://www.visualcrossing.com/weather-query-builder/Hanoi/us/last15days/), per the project requirements.
*   **Location specificity:** It was critical to use the exact location string `"Hồ Chí Minh city"` to retrieve data for the entire metropolitan area. Similar queries like "Ho Chi Minh" or "Sai Gon" pointed to more localized or slightly different coordinates, which could introduce inconsistencies.
*   **Time period:** We collected daily weather data spanning from **January 1, 2015, to October 1, 2025**. This provides a substantial ~10-year period, capturing a wide range of seasonal variations and long-term trends.
*   **Feature selection:** All available weather metrics were selected, with the logical exception of `snow` and `snowdepth`, which are irrelevant for HCMC's tropical climate.
*   **Collection strategy:**
    *   **Daily data (3,927 records):** Due to the manageable size, the daily dataset was collected using a manual download approach, requiring two accounts to bypass daily limits.
    *   **Hourly data (94,248 records):** For the much larger hourly dataset, manual collection was deemed impractical and error-prone. We developed a robust, automated Python script leveraging the Visual Crossing API.
    *   **Automated script features:** 
        *   Designed for resilience and data integrity
        *   Featuring automated API key rotation from a pool of 25+ keys from our group members *(employed some techniques to bypass account creation limit)* and automatic retries
        *   Auto-backup of the current central downloaded file before performing actions for safety measures
        *   Intelligent batch-based downloading that resumed from the last known timestamp in the central downloaded file, for convenience and maximizing the 1000 records/day/free account API key rate limit
        *   Built-in data integrity checks to detect duplicates or malformed records

$\implies$ The automated process ensured the final dataset was both complete and of high quality. The data was ready to use instantly for the daily one, and after 4 days for the hourly dataset.

<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="eda" style="display: none;">Q2. Data Understanding & Exploratory Data Analysis (EDA)</h1>
<!-- VISIBLE H1 -->
<h1 id="eda-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">Q2. Data Understanding & Exploratory Data Analysis (EDA)</h1>

We must understand the data, deeply, before we can proceed to engineer more features and starts modeling. This EDA phase serves two primary purposes:
1.  **Assess Data Quality:** To identify and understand issues like missing values, outliers, and redundancies that must be addressed during preprocessing.
2.  **Uncover Patterns & Justify Decisions:** To discover the underlying patterns, seasonality, and relationships within the data. Every significant feature engineering and modeling choice made later in this notebook will be explicitly justified by a finding from this section.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="initial-audit" style="display: none;">Q2.1. Initial Data Load & Quality Audit</h2>
<!-- VISIBLE H2 -->
<h2 id="initial-audit-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.1. Initial Data Load & Quality Audit</h2>

In [4]:
# Load the dataset from the configured path
df_raw = pd.read_csv(config.INPUT_DATA_PATH)

# Convert 'datetime' column to datetime objects and set as the index
df_raw["datetime"] = pd.to_datetime(df_raw["datetime"])
df_raw.set_index("datetime", inplace=True)

# --- Perform a high-level data audit ---
console.print("--- [bold]DataFrame Info[/bold] ---")
df_raw.info(verbose=False, memory_usage="deep")

console.print("\n--- [bold]Descriptive Statistics (Numerical Features)[/bold] ---")
display(df_raw.describe().T)

console.print("\n--- [bold]Missing Value Percentages[/bold] ---")
missing_values = df_raw.isnull().sum()
missing_percentage = (missing_values / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({
    "Missing Count": missing_values,
    "Missing Percentage": missing_percentage,
}).sort_values(by="Missing Percentage", ascending=False)

console.print(missing_df[missing_df["Missing Count"] > 0])


--- DataFrame Info ---

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3927 entries, 2015-01-01 to 2025-10-01
Columns: 37 entries, name to source
dtypes: float64(25), int64(2), object(10)
memory usage: 3.7 MB


--- Descriptive Statistics (Numerical Features) ---

,count,mean,std,min,25%,50%,75%,max
latitude,3927.0,10.776000,0.000000e+00,10.776,10.776,10.776,10.776,10.776
longitude,3927.0,106.701000,1.421266e-14,106.701,106.701,106.701,106.701,106.701
tempmax,3927.0,33.071683,1.791789e+00,24.600,32.000,33.000,34.000,39.000
tempmin,3927.0,25.136949,1.586575e+00,18.000,24.000,25.000,26.000,30.000
temp,3927.0,28.445837,1.386448e+00,22.800,27.500,28.400,29.300,33.000
feelslikemax,3927.0,38.463305,3.247412e+00,24.600,36.400,38.800,40.600,48.700
feelslikemin,3927.0,25.830736,2.943036e+00,18.000,24.000,25.000,26.000,40.000
feelslike,3927.0,31.671760,2.958532e+00,22.800,29.700,31.400,33.500,42.000
dew,3927.0,23.503896,2.242134e+00,12.700,22.400,24.200,25.000,27.900
humidity,3927.0,76.570512,9.681565e+00,49.500,69.700,77.500,84.200,99.400


--- Missing Value Percentages ---

Missing Count  Missing Percentage
severerisk           2731               69.54
preciptype            924               23.53

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="feature-defs" style="display: none;">Q2.2. Feature Definitions & Domain Context</h2>
<!-- VISIBLE H2 -->
<h2 id="feature-defs-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.2. Feature Definitions & Domain Context</h2>

| Feature Group          | Feature Name         | Data Type | Value Range (numerical) \|<br> Example Values (categorical)                               | Definition & How it may helps predicting target variable `temp`                                                                                                          |
| :--------------------- | :------------------- | :-------- | :--------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------- |
| **Identifier/Metadata**| `name`, `address`... | `object`  | `['Hồ Chí Minh city']`                                     | Constant metadata identifying the location. **Action:** Will be dropped.                                                               |
|                        | `datetime`           | `object`  | `2015-01-01` to `2025-10-01`                               | The unique timestamp for each daily record. **Action:** Will be converted to the DataFrame index.                                        |
| **Target & Temp.**     | `temp`               | `float64` | `22.8` - `33.0` (°C)                                       | **Primary Target Variable.** The average daily temperature.                                                                          |
|                        | `tempmax`/`tempmin`  | `float64` | `24.6`-`39.0` / `18.0`-`30.0` (°C)                         | The daily maximum and minimum temperatures. Key drivers of the daily average.                                                        |
|                        | `feelslike`...       | `float64` | e.g., `22.8` - `42.0` (°C)                                 | The perceived temperature, accounting for humidity and wind. Highly correlated with `temp`.                                         |
| **Atmospheric**        | `humidity`           | `float64` | `49.5` - `99.4` (%)                                        | Relative humidity. Crucial in a tropical climate as high humidity traps heat, especially impacting overnight lows.                   |
|                        | `dew`                | `float64` | `12.7` - `27.9` (°C)                                       | Dew Point. The temperature at which air becomes saturated. A direct measure of atmospheric moisture.                                   |
|                        | `sealevelpressure`   | `float64` | `1001.5` - `1016.2` (hPa)                                   | Air pressure adjusted to sea level. Changes can indicate shifting weather patterns (e.g., low pressure is associated with storms). |
|                        | `cloudcover`         | `float64` | `9.7` - `100.0` (%)                                        | The percentage of the sky covered by clouds. Directly impacts solar radiation reaching the surface.                                  |
|                        | `visibility`         | `float64` | `5.4` - `20.0` (km)                                        | The distance at which objects can be clearly seen. Often reduced by rain or high humidity.                                       |
| **Precipitation**      | `precip`             | `float64` | `0.0` - `227.2` (mm)                                       | Total daily precipitation. Defines the wet vs. dry seasons, which are central to HCMC's climate.                                   |
|                        | `precipprob`         | `int64`   | `0` or `100` (%)                                           | Probability of precipitation. In this dataset, there's only 2 values, can be understood as 0 or 1.                                                      |
|                        | `preciptype`         | `object`  | `['rain', nan]`                                            | The type of precipitation. `NaN` values deterministically correspond to `precip = 0`. **Action:** Impute `NaN` with `'none'`.          |
| **Wind**               | `windspeed`          | `float64` | `7.6` - `50.0` (km/h)                                      | The maximum sustained wind speed for the day.                                                                                        |
|                        | `windgust`           | `float64` | `8.3` - `216.0` (km/h)                                     | The highest instantaneous wind speed recorded. Can indicate storm activity.                                                          |
|                        | `winddir`            | `float64` | `0.0` - `359.7` (°)                                        | The direction from which the wind originates. Seasonal wind patterns (monsoons) are key climate drivers.                           |
| **Solar & Celestial**  | `solarradiation`     | `float64` | `0.0` - `315.8` (W/m²)                                     | The amount of solar energy reaching the surface. The primary driver of daytime temperature.                                          |
|                        | `solarenergy`        | `float64` | `0.0` - `27.2` (MJ/m²)                                     | The total solar energy over the day. Highly correlated with `solarradiation`.                                                        |
|                        | `uvindex`            | `int64`   | `0` - `10`                                                 | The strength of ultraviolet radiation. A proxy for sun intensity and lack of cloud cover.                                            |
|                        | `sunrise`/`sunset`   | `object`  | `2015-01-01T06:11:22`                                      | Timestamps for sunrise and sunset. **Action:** Can be engineered into a `daylight_hours` feature.                                      |
|                        | `moonphase`          | `float64` | `0.0` (new) - `0.98` (waning)                              | The fraction of the moon that is illuminated. May have a subtle, long-term cyclical influence.                                       |
| **Text & Categorical** | `conditions`         | `object`  | `['Partially cloudy', 'Rain...']`                          | A high-level summary of the day's weather.                                                                                           |
|                        | `description`        | `object`  | `['Partly cloudy throughout...']`                          | A more detailed, human-readable description of the weather. **Action:** Will be processed using NLP techniques.                      |
|                        | `icon`               | `object`  | `['partly-cloudy-day', 'rain']`                            | A machine-readable categorical summary of conditions.                                                                                |
| **Risk**               | `severerisk`         | `float64` | `5.0` - `75.0`                                             | A numerical rating for the risk of severe weather events. `NaN` indicates no recorded risk. **Action:** Impute `NaN` with `0`.      |

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="key-predictors" style="display: none;">Q2.3. Deep Dive: Key Predictive Feature Groups</h2>
<!-- VISIBLE H2 -->
<h2 id="key-predictors-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.3. Deep Dive: Key Predictive Feature Groups</h2>

### 1. The Solar Radiation & Cloud Cover Nexus
*   **Physical Principle:** Solar radiation is the primary engine of Earth's climate, representing the incoming energy that heats the surface. Cloud cover acts as the primary regulator of this energy, reflecting a significant portion back into space before it can be absorbed.
*   **Measurement:**
    *   `solarradiation` is measured in Watts per square meter (W/m²), quantifying the instantaneous power of solar energy reaching a given area.
    *   `cloudcover` is a percentage, often estimated from satellite imagery or ground-based observations.
*   **Predictive Hypothesis:**
    *   We hypothesize a **strong positive correlation** between `solarradiation` and `tempmax`. Days with high, direct solar radiation will almost certainly be the hottest.
    *   Conversely, we expect a **strong negative correlation** between `cloudcover` and `temp`. Higher cloud cover will suppress daytime heating, leading to cooler average temperatures. This relationship is fundamental to HCMC's climate, where the difference between a clear day in the dry season and an overcast day in the rainy season is pronounced.

### 2. The Atmospheric Moisture Duo: Humidity & Dew Point
*   **Physical Principle:** These two features quantify the amount of water vapor in the atmosphere, which plays a crucial role in temperature regulation via the greenhouse effect. Water vapor traps outgoing longwave radiation, preventing heat from escaping, especially at night.
*   **Measurement:**
    *   `humidity` is *relative*—it's the percentage of water vapor in the air compared to the maximum amount it *could* hold at that temperature.
    *   `dew` (Dew Point) is an *absolute* measure. It is the temperature at which the air would become 100% saturated. A higher dew point means there is more actual moisture in the air, regardless of the current temperature.
*   **Predictive Hypothesis:**
    *   In HCMC's consistently warm climate, the **Dew Point will be a more robust predictor than Relative Humidity**.
    *   We expect a **strong positive correlation between `dew` and `tempmin` (overnight low)**. High atmospheric moisture (high dew point) acts like a blanket, preventing nighttime temperatures from dropping significantly. This is a key reason why humid nights in HCMC feel so warm.

### 3. The Precipitation System: A Cooling & Lagged Effect
*   **Physical Principle:** Precipitation has multiple effects on temperature. The act of evaporation from wet surfaces actively cools the air (evaporative cooling). Additionally, the cloud systems that produce rain block solar radiation.
*   **Measurement:** `precip` is the total volume of rainfall, measured in millimeters (mm).
*   **Predictive Hypothesis:**
    *   The immediate effect of `precip` on the same day's `temp` will be **negative**. Rainy days are cooler days.
    *   More importantly, we hypothesize a **lagged effect**. A day with heavy rainfall will increase ground moisture, leading to higher humidity and dew points on subsequent days. Therefore, `precip` from a few days prior may positively influence the `tempmin` of the current day. This justifies the creation of lagged and rolling-sum precipitation features.

### 4. Wind Dynamics: Direction & Speed
*   **Physical Principle:** Wind influences temperature through advection (transporting air masses) and evaporative cooling.
*   **Measurement:**
    *   `windspeed` (km/h) measures the rate of air movement.
    *   `winddir` (degrees) indicates the origin direction of the wind.
*   **Predictive Hypothesis:**
    *   `windspeed` will have a modest cooling effect, especially on the `feelslike` temperature.
    *   **`winddir` is potentially a powerful but complex feature.** In HCMC, wind direction is strongly tied to the two monsoon seasons. The Southwest Monsoon (May-Nov) brings moist, rain-bearing winds from the ocean, while the Northeast Monsoon (Dec-Apr) brings drier, cooler air from the continent. We hypothesize that `winddir` can act as a strong seasonal indicator, but its circular nature (359° is close to 0°) means it must be transformed into cyclical (sine/cosine) features to be used effectively by the model.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="target-analysis" style="display: none;">Q2.4. Target Variable Analysis: Trends & Seasonality</h2>
<!-- VISIBLE H2 -->
<h2 id="target-analysis-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.4. Target Variable Analysis: Trends & Seasonality</h2>

In [5]:
# fig_target_timeseries = eda.plot_target_variable_time_series(df=df_raw, target_col=config.TARGET_VARIABLE)
# fig_target_timeseries.show()


*   **1. Strong & Predictable Annual Seasonality:**
    *   The **30-day rolling mean (blue line)** clearly reveals a consistent, repeating annual cycle of peaks and troughs. This strong seasonal pattern is the most dominant signal in the data.
    *   This observation confirms our hypothesis that time-based features are essential. The model must be given information about the time of year to capture this cycle.

*   **2. Evidence of a Long-Term Upward Trend:**
    *   The **365-day rolling mean (red line)** filters out the seasonal fluctuations, exposing a subtle but persistent upward drift in the baseline temperature over the 10-year period.
    *   This trend, likely reflecting broader climate change patterns, indicates that the data is **non-stationary**. A simple model that only learns seasonality would degrade over time. Our model must also account for this long-term evolution.

*   **3. Fluctuating Volatility (Noise):**
    *   The lower panel, showing the rolling standard deviation, quantifies the data's volatility. The temperature is not equally predictable year-round.
    *   We can observe periods of higher volatility (wider swings in temperature), particularly during the transitional periods between the dry and wet seasons. This suggests that the model's accuracy may naturally vary depending on the time of year.

$\implies$ Based on these three distinct components (seasonality, trend, noise), we establish the following non-negotiable feature engineering requirements:

*   **To capture seasonality:** We will engineer **cyclical features** (e.g., sine/cosine transformations of `day_of_year` and `month`). These allow the model to understand the cyclical nature of time, where December is "close" to January.
*   **To capture trend:** We will include a **`year` feature** or other trend-aware components to allow the model to account for the gradual increase in baseline temperature over time.
*   **To capture recent history & noise:** We will create **lagged features** (e.g., temperature from the previous day) and **rolling window features** (e.g., the average temperature over the last 7 days). These provide the model with a "memory" of the recent system state, helping it to predict short-term fluctuations around the seasonal and trend lines.

Next, we will perform a deeper dive into the seasonal component to better understand the specific climate dynamics of Ho Chi Minh City throughout the year.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="seasonal-deep-dive" style="display: none;">Q2.5. Seasonal Pattern Deep Dive: The Monsoon Cycle</h2>
<!-- VISIBLE H2 -->
<h2 id="seasonal-deep-dive-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.5. Seasonal Pattern Deep Dive: The Monsoon Cycle</h2>

In [6]:
# fig_monthly_seasonality = eda.plot_monthly_seasonality(df=df_raw, target_col=config.TARGET_VARIABLE)
# fig_monthly_seasonality.show()


*   **Primary Peak (Pre-monsoon Heat):** The hottest period of the year occurs in **April and May**. This is not mid-summer but rather the end of the dry season, a period characterized by intense, building heat and humidity just before the onset of the rainy Southwest Monsoon.

*   **Mid-year Dip (Monsoon Cooling):** A noticeable dip in the median temperature and a compression of the temperature range occurs from **June through August**. This corresponds to the peak of the rainy season. The increased cloud cover and frequent rainfall have a significant cooling effect, suppressing daytime maximum temperatures.

*   **Coolest Period (Dry Season):** The coolest months are **December and January**. This coincides with the Northeast Monsoon, which brings comparatively drier and cooler air masses to the region.

*   **Outlier Confirmation:** The plot also provides clear visual confirmation of low-temperature outliers, particularly in the cooler months of December and January. These represent unusual cold snaps that a robust model must be able to handle without its feature scaling being skewed.

$\implies$ **Feature Engineering:**
1.  This complex, non-sinusoidal pattern suggests creating **cyclical features** (`sin_day_of_year`, `cos_day_of_year`). A simple numerical `month` feature would be insufficient, as it cannot learn that May is climatically closer to June than it is to December.
2.  The presence of outliers provides further, conclusive evidence for our decision to use `RobustScaler` during preprocessing.

To mathematically isolate these components, we will now perform an STL (Seasonal-Trend-Loess) decomposition.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="stl-decomposition" style="display: none;">Q2.6. STL Decomposition: Isolating Key Signals</h2>
<!-- VISIBLE H2 -->
<h2 id="stl-decomposition-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.6. STL Decomposition: Isolating Key Signals</h2>

In [7]:
# fig_stl = eda.plot_stl_decomposition(df=df_raw, target_col=config.TARGET_VARIABLE)
# fig_stl.show()


The STL decomposition provides a mathematical separation of the time series, allowing us to analyze each component in isolation. We use an **additive model** (`Observed = Trend + Seasonal + Residual`) because the magnitude of the seasonal swings does not appear to change proportionally with the long-term trend; the annual temperature variation is relatively constant.

*   **Trend Component:**
    *   This panel confirms the non-stationary nature of the data with a clear, non-linear trend.
    *   The baseline temperature shows a distinct rise and fall over multi-year periods, with a notable acceleration in the upward trend from mid-2023 onwards.
    *   **Modeling Implication:** This complex trend confirms that a simple linear `year` feature will be insufficient. Our model must be flexible enough to capture this dynamic baseline, reinforcing the value of tree-based models and rich temporal features.

*   **Seasonal Component:**
    *   This successfully isolates the strong, repeating annual pattern, which is the dominant driver of temperature variation.
    *   As observed, the pattern is highly consistent through the middle of the dataset (approx. 2016-2024). The slight variations in amplitude at the very beginning and end of the series could be attributed to specific climate phenomena in those years or edge effects of the decomposition algorithm.
    *   **Modeling Implication:** This component's stability and strength underscore the critical importance of the cyclical features we plan to engineer.

*   **Residual Component:**
    *   This represents the "noise" or randomness left after the predictable trend and seasonal components have been removed.
    *   Crucially, the residuals are centered around zero and display no obvious leftover patterns. This indicates that the decomposition has effectively captured the primary signals.
    *   **Modeling Implication:** The residuals represent the day-to-day weather fluctuations our model will aim to predict using features like recent lags, rolling windows, and atmospheric conditions (`cloudcover`, `precip`, etc.).

$\implies$ **Conclusion:** The decomposition validates our initial analysis. The temperature is a composite of a dynamic long-term trend, a powerful seasonal cycle, and short-term noise. Our feature engineering strategy must explicitly provide the model with tools to address each of these components individually.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="feature-distribution" style="display: none;">Q2.7. Predictor Analysis: Distribution, Skew, & Outliers</h2>
<!-- VISIBLE H2 -->
<h2 id="feature-distribution-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.7. Predictor Analysis: Distribution, Skew, & Outliers</h2>

Having thoroughly analyzed the target variable, we now turn to the key predictors. Understanding their statistical distributions is essential for making sound preprocessing decisions. Features with significant skew or extreme outliers can disproportionately influence certain models and require robust handling.

In [8]:
# Select a representative subset of key numerical features - a mix of atmospheric, precipitation, and solar variables
features_to_plot = [
    "humidity",
    "dew",
    "sealevelpressure",
    "cloudcover",
    "solarradiation",
    "uvindex",
    "precip",
    "windspeed",
    "windgust",
]

feature_color_map = {
    feature: pio.templates["te"].layout.colorway[i % len(pio.templates["te"].layout.colorway)]
    for i, feature in enumerate(features_to_plot)
}

# fig_distributions = eda.plot_feature_distributions(df=df_raw, features=features_to_plot, color_map=feature_color_map)
# fig_distributions.show()


We can categorize the features into three distinct groups:

*   **1. Approximately Symmetric Distributions:**
    *   `Humidity`, `Sealevelpressure`, and `Cloudcover` exhibit largely symmetric, well-behaved distributions. For these features, the mean (dashed line) is very close to the median (solid line), indicating a lack of significant skew. While some have outliers on both ends, they don't appear to heavily distort the central tendency.

*   **2. Moderately Skewed Distributions:**
    *   `Dew`, `Solarradiation`, and `Windspeed` show a noticeable skew. `Dew` and `Solarradiation` are moderately left-skewed (a longer tail towards lower values), while `Windspeed` is moderately right-skewed. This asymmetry is confirmed by the divergence of their mean and median values.

*   **3. Extremely Skewed, Outlier-Dominated Distributions:**
    *   `Precip` and `Windgust` represent the most challenging features. The distributions are heavily compressed near zero (for `Precip`), with long, thin tails representing rare but extreme events (heavy rainfall, strong gusts). The vast majority of data points are concentrated in a very small range, with a few extreme outliers dramatically extending the feature's scale. The `uvindex` also shows a strong left skew, compounded by its discrete, integer-based nature.

$\implies$ **Conclusion on Scaling Strategy:**
Using a standard `mean-and-variance` scaler (`StandardScaler`) would be methodologically unsound. The extreme outliers in `Precip` and `Windgust` would massively distort the mean and standard deviation, effectively "squashing" the informational variance of the other 99% of data points.

Therefore, the use of `RobustScaler` is better suited. By scaling data based on the **median and interquartile range (IQR)**, `RobustScaler` is inherently resistant to the influence of extreme outliers, ensuring that all features are scaled on a consistent and meaningful basis.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="bivariate-analysis" style="display: none;">Q2.8. Bivariate & Seasonal Relationship Analysis</h2>
<!-- VISIBLE H2 -->
<h2 id="bivariate-analysis-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q2.8. Bivariate & Seasonal Relationship Analysis</h2>

In [9]:
pairplot_features = [
    "temp",
    "dew",
    "precip",
    "humidity",
    "windspeed",
    "solarradiation",
    "cloudcover",
]

# fig_pairplot = eda.plot_seasonal_pairplot(df=df_raw, features=pairplot_features)
# fig_pairplot.show()


*   **1. Diagonals Confirm Strong Seasonal Shifts:**
    *   The seasonal density plots on the diagonal clearly show that the distributions of key variables change dramatically between seasons.
    *   During the **Rainy Season (blue)**, `Dew`, `Humidity`, and `Cloudcover` are all significantly higher and more concentrated than in the **Dry Season (yellow)**. `Precipitation` shifts from being almost entirely zero to having a long tail of rain events.
    *   This confirms these variables are powerful indicators of the prevailing monsoon season and will be critical for the model.

*   **2. Solar Radiation & Cloud Cover: The Primary Drivers:**
    *   The scatter plots for `Temp` vs. `Solarradiation` and `Temp` vs. `Cloudcover` show the strongest, most consistent relationships.
    *   There is a clear positive linear trend between `Solarradiation` and `Temp`: more sun results in higher temperatures.
    *   Conversely, there is a strong negative linear trend between `Cloudcover` and `Temp`: more cloud cover leads to cooler temperatures.

*   **3. The Less Pronounced Role of Humidity and Dew Point:**
    *   The `Temp` vs. `Dew` plot shows a positive relationship; higher absolute moisture is associated with higher temperatures, likely due to the "blanket effect" trapping heat.
    *   The `Temp` vs. `Humidity` plot reveals a more complex, negative relationship. This seems counter-intuitive but is explained by the seasonal context: the highest humidity occurs during the rainy season, which is also when increased cloud cover and rain suppress daytime temperatures. This highlights that `Humidity`'s effect is often indirect (i.e. non-linear).

*   **4. Visual Confirmation of Multicollinearity:**
    *   The relationship between `Dew` and `Humidity` is very strong and linear, as is the inverse relationship between `Solarradiation` and `Cloudcover`.
    *   This finding is critical for modeling. While a tree-based model can handle this, a linear model would struggle with such highly correlated predictors. We should create distinct feature sets later in the pipeline.

$\implies$ **EDA Conclusion:** Now we understand the data much better, and can make more informed decisions when processing the data, engineering the features, and modeling later on. 

<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="processing" style="display: none;">Q3. Data Processing & Correlation Analysis</h1>
<!-- VISIBLE H1 -->
<h1 id="processing-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">Q3. Data Processing & Correlation Analysis</h1>

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="sanitization" style="display: none;">Q3.1. Column Sanitization</h2>
<!-- VISIBLE H2 -->
<h2 id="sanitization-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q3.1. Column Sanitization</h2>

Our first processing step is to remove all columns that were identified as non-predictive during the EDA. This includes:
*   **Constant Identifiers:** Columns like `name`, `address`, `latitude`, etc., which have the same value for every record.
*   **Redundant Duplicates:** Columns that are exact copies of others, such as `windspeedmax` being identical to `windspeed`.
*   **Purely Descriptive Text:** Columns like `description` that will be handled by more targeted feature engineering later, not kept in their raw form.
*   **Time-related Objects:** `sunrise` and `sunset` will be engineered into more useful numerical features (like day length) and are not needed in their raw object format.

This step reduces memory usage and focuses our dataset on potentially valuable features.

In [10]:
df_processed = df_raw.copy()

cols_to_drop = [
    # Constant Identifiers
    "name",
    "address",
    "resolvedAddress",
    "latitude",
    "longitude",
    "source",
    # Duplicate
    "windspeedmax",
    # Descriptive/Object columns to be engineered later
    "description",
    "conditions",
    "icon",
    "sunrise",
    "sunset",
    # Redundant energy column
    "solarenergy",
]

shape_before = df_processed.shape

df_processed.drop(columns=cols_to_drop, inplace=True, errors="ignore")

shape_after = df_processed.shape

console.print(f"DataFrame shape before sanitization: [yellow]{shape_before}[/yellow]")
console.print(f"DataFrame shape after sanitization:  [green]{shape_after}[/green]")
console.print(f"Columns removed: [red]{shape_before[1] - shape_after[1]}[/red]")

console.print("\n[bold]Remaining columns in the processed DataFrame:[/bold]")
console.print(df_processed.columns.tolist())


DataFrame shape before sanitization: (3927, 37)

DataFrame shape after sanitization:  (3927, 24)

Columns removed: 13

Remaining columns in the processed DataFrame:

[
    'tempmax',
    'tempmin',
    'temp',
    'feelslikemax',
    'feelslikemin',
    'feelslike',
    'dew',
    'humidity',
    'precip',
    'precipprob',
    'precipcover',
    'preciptype',
    'windgust',
    'windspeed',
    'windspeedmean',
    'windspeedmin',
    'winddir',
    'sealevelpressure',
    'cloudcover',
    'visibility',
    'solarradiation',
    'uvindex',
    'severerisk',
    'moonphase'
]

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="imputation" style="display: none;">Q3.2. Missing Value Imputation</h2>
<!-- VISIBLE H2 -->
<h2 id="imputation-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q3.2. Missing Value Imputation</h2>

The EDA revealed that missing values are not random but systematic. We address them with deterministic, rule-based imputation that preserves the underlying meaning of the data. Drop is no-no:)

*   **`preciptype`:** Missingness in this column occurs exclusively when the `precip` (precipitation amount) is zero. Therefore, we will fill these `NaN` values with the string `'none'`. This transforms the missing data into an explicit, meaningful category for "no-precipitation days."
*   **`severerisk`:** Values in this column are only recorded when a certain risk threshold is met. The absence of a value implies the absence of risk. We will fill these `NaN` values with `0`.

This approach avoids data deletion and ensures that every value in our dataset is explicitly defined, and potentially providing more predictive signals to the models.

In [11]:
# 1. Fill 'preciptype' based on the deterministic relationship with 'precip'
df_processed["preciptype"].fillna("none", inplace=True)

# 2. Fill 'severerisk' with 0, as missingness implies no recorded severe risk
df_processed["severerisk"].fillna(0, inplace=True)


remaining_nulls = df_processed.isnull().sum().sum()

null_count_color = "green" if remaining_nulls == 0 else "red"
console.print(
    f"Total remaining null values in the DataFrame: [bold {null_count_color}]{remaining_nulls}[/bold {null_count_color}]"
)

if remaining_nulls == 0:
    console.print("\n[bold]Value counts for 'preciptype' after imputation:[/bold]")
    print(df_processed["preciptype"].value_counts())

    console.print("\n[bold]Value counts for 'severerisk' after imputation:[/bold]")
    print(df_processed["severerisk"].value_counts())

else:
    console.print("There are still missing values that need to be addressed.")


Total remaining null values in the DataFrame: 0

Value counts for 'preciptype' after imputation:

preciptype
rain    3003
none     924
Name: count, dtype: int64


Value counts for 'severerisk' after imputation:

severerisk
0.0     2731
30.0     546
10.0     448
60.0     195
75.0       6
5.0        1
Name: count, dtype: int64


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="correlation-deep-dive" style="display: none;">Q3.3. Correlations Check</h2>
<!-- VISIBLE H2 -->
<h2 id="correlation-deep-dive-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q3.3. Correlations Check</h2>

We will employ two different correlation methods to ensure a comprehensive understanding:

*   **Pearson Correlation (`r`):**
    *   Standard method, measuring the strength of the *linear* relationship between two variables
    *   Sensitive to outliers
    *   A value of +1 indicates a perfect positive linear relationship, -1 a perfect negative linear relationship, and 0 indicates no linear relationship.

*   **Spearman Correlation (`ρ` or `rho`):**
    *   Measures the strength of the *monotonic* relationship between two variables
    *   Operates on the ranks of the data rather than the raw values $\rightarrow$ highly robust to outliers, capable of capturing relationships that are consistently increasing or decreasing but not necessarily linear.

Our weather data has outliers, and potential non-linearities **$\implies$** the **Spearman correlation provides a more reliable and robust assessment** of the underlying relationships.

In [12]:
df_numerical = df_processed.select_dtypes(include=np.number)

# fig_pearson = eda.plot_correlation_heatmap_enhanced(df=df_numerical, method="pearson")
# fig_pearson.show()

ah.display_correlation_summary(df=df_numerical, target=config.TARGET_VARIABLE, method="pearson")


Target Correlation Summary (Pearson) 

  Most Positively Correlated                            Most Negatively Correlated with                            
         with 'temp'                                                 'temp'                                        
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓                         ┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓                          
┃ Feature      ┃ Coefficient ┃                         ┃ Feature          ┃ Coefficient ┃                          
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩                         ┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩                          
│ feelslike    │       0.893 │                         │ humidity         │      -0.340 │                          
│ tempmax      │       0.847 │                         │ cloudcover       │      -0.308 │                          
│ tempmin      │       0.792 │                         │ precip           │      -0.222 │                          
│ feelslikemin │       0.771 │                         │ sealevelpressure │      -0.128 │                          
│ feelslikemax │       0.634 │                         │ winddir          │      -0.087 │                          
└──────────────┴─────────────┘                         └──────────────────┴─────────────┘

Predictor Multicollinearity Summary (Pearson) 

       Highest Positive Predictor Pairs                           Highest Negative Predictor Pairs                 
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓            ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓         
┃ Feature 1      ┃ Feature 2    ┃ Coefficient ┃            ┃ Feature 1  ┃ Feature 2        ┃ Coefficient ┃         
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩            ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩         
│ tempmin        │ feelslikemin │       0.911 │            │ dew        │ sealevelpressure │      -0.652 │         
│ solarradiation │ uvindex      │       0.911 │            │ cloudcover │ solarradiation   │      -0.601 │         
│ temp           │ feelslike    │       0.893 │            │ humidity   │ solarradiation   │      -0.567 │         
│ tempmax        │ temp         │       0.847 │            │ cloudcover │ uvindex          │      -0.557 │         
│ feelslikemax   │ feelslike    │       0.833 │            │ humidity   │ sealevelpressure │      -0.554 │         
└────────────────┴──────────────┴─────────────┘            └────────────┴──────────────────┴─────────────┘

In [13]:
# fig_spearman = eda.plot_correlation_heatmap_enhanced(df=df_numerical, method="spearman")
# fig_spearman.show()

ah.display_correlation_summary(df=df_numerical, target=config.TARGET_VARIABLE, method="spearman")


Target Correlation Summary (Spearman) 

Most Positively Correlated with                           Most Negatively Correlated with                          
             'temp'                                                    'temp'                                      
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓                         ┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓                        
┃ Feature        ┃ Coefficient ┃                         ┃ Feature          ┃ Coefficient ┃                        
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩                         ┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩                        
│ feelslike      │       0.874 │                         │ humidity         │      -0.393 │                        
│ tempmax        │       0.840 │                         │ cloudcover       │      -0.335 │                        
│ tempmin        │       0.757 │                         │ precip           │      -0.160 │                        
│ feelslikemin   │       0.756 │                         │ winddir          │      -0.101 │                        
│ solarradiation │       0.599 │                         │ sealevelpressure │      -0.091 │                        
└────────────────┴─────────────┘                         └──────────────────┴─────────────┘

Predictor Multicollinearity Summary (Spearman) 

       Highest Positive Predictor Pairs                           Highest Negative Predictor Pairs                 
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓            ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓         
┃ Feature 1      ┃ Feature 2    ┃ Coefficient ┃            ┃ Feature 1  ┃ Feature 2        ┃ Coefficient ┃         
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩            ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩         
│ tempmin        │ feelslikemin │       0.999 │            │ dew        │ sealevelpressure │      -0.610 │         
│ solarradiation │ uvindex      │       0.881 │            │ humidity   │ solarradiation   │      -0.600 │         
│ temp           │ feelslike    │       0.874 │            │ cloudcover │ solarradiation   │      -0.584 │         
│ tempmax        │ temp         │       0.840 │            │ humidity   │ visibility       │      -0.566 │         
│ feelslikemax   │ feelslike    │       0.823 │            │ humidity   │ uvindex          │      -0.559 │         
└────────────────┴──────────────┴─────────────┘            └────────────┴──────────────────┴─────────────┘

*   **1. Key drivers:**
    *   Both matrices confirm that `temp` is most strongly correlated with its variants (`feelslike`, `tempmax`, `tempmin`), which is expected.
    *   More importantly, they both confirm our hypotheses about the primary physical drivers: `solarradiation` has a strong positive correlation with temperature, while `humidity` and `cloudcover` have the strongest negative correlations.

*   **2. Severe multicollinearity:** Both methods reveal two critical clusters of extreme multicollinearity, which is a concern for linear models:
    *   **Temperature cluster:** Features like `temp`, `feelslike`, `tempmin`, and `feelslikemin` are all highly inter-correlated (many coefficients > 0.80).
    *   **Humidity cluster:** `dew` and `humidity` show a very strong positive relationship (`r` = 0.82, `ρ` = 0.80).
    *   **Solar cluster:** `solarradiation` and `uvindex` are also highly correlated, effectively measuring the same underlying phenomenon of sun intensity.

*   **3. Spearman's robustness:**
    *   The correlation between `tempmin` and `feelslikemin` is **`ρ` = 0.999**. This indicates they are functionally identical in terms of their rank order and are perfectly redundant predictors.
    *   The negative correlations for `humidity` and `cloudcover` with `temp` are **stronger** in the Spearman results $\rightarrow$ increases confidence in their importance, as their relationship is consistently monotonic,  not just an artifact of a few outliers.

$\implies$ Attempting to fit a standard linear model with all of these features would result in **unstable and uninterpretable** coefficients. We need another approach to manage this multicollinearity.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="multicollinearity-strategy" style="display: none;">Q3.4. Strategy for Multicollinearity</h2>
<!-- VISIBLE H2 -->
<h2 id="multicollinearity-strategy-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q3.4. Strategy for Multicollinearity</h2>

We have confirmed the presence of severe multicollinearity - this occurs when two or more predictor variables are highly correlated, making it difficult for a model to disentangle their individual effects on the target variable. For standard **linear models**, this can lead to **unstable, unreliable, and uninterpretable coefficient** estimates.

*   **Standard diagnostic tool: Variance Inflation Factor (VIF)**
    *   We can use this common method to diagnose, which calculates how much the variance of a regression coefficient is inflated due to its correlation with other predictors.
    *   A high VIF (usually > 5 or 10) for a feature indicates that it is highly collinear and a candidate for removal.

*   **Our choice: Regularization**
    *   While a manual, VIF-based feature pruning is a valid approach, we will instead use a more integrated strategy by leveraging models with **built-in regularization**.
    *   Algorithms like `RidgeCV`, `ElasticNetCV`, and `MultiTaskLassoCV` (which we will use for feature selection) are specifically designed to handle multicollinearity. They add a penalty term to the loss function that shrinks the coefficients of correlated predictors.
    *   **In practice, this means the model itself will learn to "de-emphasize" redundant features.** For example, when faced with the highly correlated `tempmin` and `feelslikemin`, a regularized model will likely assign a significant coefficient to one while shrinking the coefficient of the other towards zero. This achieves the same goal as manual VIF-based removal but in a data-driven, automated fashion that is integrated directly into the model training process.

$\implies$ **Justification:** By choosing regularized models and embedded feature selection, we have a more robust and sophisticated approach to managing multicollinearity. This allows the model to make the optimal data-driven trade-offs during training.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="finalizing-df" style="display: none;">Q3.5. Final Processed DataFrame before Feature Engineering</h2>
<!-- VISIBLE H2 -->
<h2 id="finalizing-df-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q3.5. Final Processed DataFrame before Feature Engineering</h2>

In [14]:
console.print(
    f"[bold]Final Shape of Processed DataFrame:[/bold] [green]{df_processed.shape}[/green]",
    "\n[bold]Data Types and Memory Usage:[/bold]",
)
df_processed.info(memory_usage="deep")


Final Shape of Processed DataFrame: (3927, 24) 
Data Types and Memory Usage:

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3927 entries, 2015-01-01 to 2025-10-01
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tempmax           3927 non-null   float64
 1   tempmin           3927 non-null   float64
 2   temp              3927 non-null   float64
 3   feelslikemax      3927 non-null   float64
 4   feelslikemin      3927 non-null   float64
 5   feelslike         3927 non-null   float64
 6   dew               3927 non-null   float64
 7   humidity          3927 non-null   float64
 8   precip            3927 non-null   float64
 9   precipprob        3927 non-null   int64  
 10  precipcover       3927 non-null   float64
 11  preciptype        3927 non-null   object 
 12  windgust          3927 non-null   float64
 13  windspeed         3927 non-null   float64
 14  windspeedmean     3927 non-null   float64
 15  windspeedmin      3927 non-null   float64
 16  winddir           3927 n

<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="feature-engineering" style="display: none;">Q4. Feature Engineering</h1>
<!-- VISIBLE H1 -->
<h1 id="feature-engineering-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">Q4. Feature Engineering</h1>

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-temporal" style="display: none;">Q4.1. Foundational Temporal Features</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-temporal-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.1. Foundational Temporal Features</h2>

One of the most critical stages of the project - we need to transform the clean, processed data into an information-rich feature set that makes the underlying patterns we discovered in the EDA explicit for our machine learning models. We will engineer features to encode four key dimensions of the time-series data:

1.  **Basic timestamps:** Discrete markers of time (`year`, `month`).
2.  **Cyclical seasonality:** Continuous representations of the annual cycle.
3.  **Complex seasonality (Fourier):** Features to capture multi-frequency patterns like the bimodal climate cycle.
4.  **Domain-specific indicators:** Meteorological features that make physical relationships explicit.

In [15]:
df_engineered = df_processed.copy()

# --- 4.1.2: Basic Temporal Features ---
# These features help the model understand the long-term trend and basic monthly groupings.
df_engineered["year"] = df_engineered.index.year
df_engineered["month"] = df_engineered.index.month
df_engineered["week_of_year"] = df_engineered.index.isocalendar().week.astype(int)
df_engineered["day_of_year"] = df_engineered.index.dayofyear

# --- 4.1.3: Cyclical Features ---
# Sine/cosine transformations are essential for cyclical data. They encode the
# "closeness" of time points (e.g., Dec 31 is close to Jan 1) in a way that
# linear models can understand.
day_of_year_max = 366.0  # Account for leap years
month_max = 12.0

df_engineered["sin_day_of_year"] = np.sin(2 * np.pi * df_engineered["day_of_year"] / day_of_year_max)
df_engineered["cos_day_of_year"] = np.cos(2 * np.pi * df_engineered["day_of_year"] / day_of_year_max)
df_engineered["sin_month"] = np.sin(2 * np.pi * df_engineered["month"] / month_max)
df_engineered["cos_month"] = np.cos(2 * np.pi * df_engineered["month"] / month_max)

# --- 4.1.4: Fourier Series Features ---
# Our EDA revealed a bimodal seasonal pattern. While the annual sin/cos
# captures the main cycle, higher-frequency Fourier terms can help model this
# more complex, multi-frequency seasonality (e.g., a semi-annual cycle). Just maybe, but hey it doesn't hurt to try.
for k in range(2, 4):  # Add k=2 (semi-annual) and k=3 (quarterly) terms
    df_engineered[f"fourier_sin_{k}"] = np.sin(2 * np.pi * k * df_engineered["day_of_year"] / day_of_year_max)
    df_engineered[f"fourier_cos_{k}"] = np.cos(2 * np.pi * k * df_engineered["day_of_year"] / day_of_year_max)


temporal_features_added = [
    "year",
    "month",
    "week_of_year",
    "day_of_year",
    "sin_day_of_year",
    "cos_day_of_year",
    "sin_month",
    "cos_month",
    "fourier_sin_2",
    "fourier_cos_2",
    "fourier_sin_3",
    "fourier_cos_3",
]
console.print(
    "[bold]Temporal features created.[/bold]\n",
    f" - Number of new features: [green]{len(temporal_features_added)}[/green]\n",
    f" - DataFrame shape is now: [green]{df_engineered.shape}[/green]",
)

console.print("\n[bold]Sample of new temporal features:[/bold]")
display(df_engineered[temporal_features_added].head())


Temporal features created.
  - Number of new features: 12
  - DataFrame shape is now: (3927, 36)

Sample of new temporal features:

,year,month,week_of_year,day_of_year,sin_day_of_year,cos_day_of_year,sin_month,cos_month,fourier_sin_2,fourier_cos_2,fourier_sin_3,fourier_cos_3
datetime,,,,,,,,,,,,
2015-01-01,2015,1,1,1,0.017166,0.999853,0.5,0.866025,0.034328,0.999411,0.051479,0.998674
2015-01-02,2015,1,1,2,0.034328,0.999411,0.5,0.866025,0.068615,0.997643,0.102821,0.994700
2015-01-03,2015,1,1,3,0.051479,0.998674,0.5,0.866025,0.102821,0.994700,0.153891,0.988088
2015-01-04,2015,1,1,4,0.068615,0.997643,0.5,0.866025,0.136906,0.990584,0.204552,0.978856
2015-01-05,2015,1,2,5,0.085731,0.996318,0.5,0.866025,0.170830,0.985301,0.254671,0.967028


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-domain" style="display: none;">Q4.2. Domain-Specific Feature Creation</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-domain-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.2. Domain-Specific Feature Creation</h2>

Beyond generic time-based signals, we can engineer features that make the physical relationships we hypothesized in our EDA explicit. By calculating these values, we provide the model with pre-computed, high-value predictors that directly relate to the drivers of temperature in a tropical climate.

In [16]:
# --- 4.2.1: Daylight Hours ---
# This is a more direct measure of solar energy potential than day_of_year.
sunrise_sunset = pd.read_csv(
    config.INPUT_DATA_PATH, usecols=["datetime", "sunrise", "sunset"], parse_dates=["datetime", "sunrise", "sunset"]
).set_index("datetime")

df_engineered["daylight_hours"] = (sunrise_sunset["sunset"] - sunrise_sunset["sunrise"]).dt.total_seconds() / 3600

# --- 4.2.2: Daily Temperature Range ---
# This feature quantifies daily temperature volatility. A low range often indicates
# a cloudy or rainy day, while a high range suggests a clear, sunny day.
df_engineered["temp_range"] = df_engineered["tempmax"] - df_engineered["tempmin"]

# --- 4.2.3: Dew Point Deficit ---
# This measures how close the air is to saturation. A smaller deficit means
# higher relative humidity and can inhibit nighttime cooling.
df_engineered["dew_point_deficit"] = df_engineered["temp"] - df_engineered["dew"]


domain_features_added = ["daylight_hours", "temp_range", "dew_point_deficit"]
console.print(
    "[bold]Domain-specific features created.[/bold]\n",
    f" - Number of new features: [green]{len(domain_features_added)}[/green]\n",
    f" - DataFrame shape is now: [green]{df_engineered.shape}[/green]",
)

console.print("\n[bold]Sample of new domain-specific features:[/bold]")
display(df_engineered[domain_features_added].head())


Domain-specific features created.
  - Number of new features: 3
  - DataFrame shape is now: (3927, 39)

Sample of new domain-specific features:

,daylight_hours,temp_range,dew_point_deficit
datetime,,,
2015-01-01,11.504722,8.0,8.7
2015-01-02,11.507222,10.0,9.2
2015-01-03,11.509722,9.0,7.8
2015-01-04,11.512500,8.0,5.8
2015-01-05,11.515556,5.7,4.6


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-lag-rolling" style="display: none;">Q4.3. Lag & Rolling Window Features</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-lag-rolling-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.3. Lag & Rolling Window Features</h2>

These features are the cornerstone of time-series forecasting, allowing the model to learn from the recent past.

*   **Lag features** provide a direct "memory" of a variable's value from previous days. For example, the temperature from yesterday (`temp_lag_1`) is one of the single best predictors for the temperature today. We will test just how much it is with a naive baseline (persistence) model later.

*   **Rolling window features** provide "context" by summarizing a variable's behavior over a recent period. This captures momentum and volatility. For example, the 7-day average temperature (`temp_roll_7d_mean`) tells the model if the past week has been generally hotter or cooler than the seasonal average.

**Preventing data leakage:**
* To calculate rolling window features for a given day `t`, we **only use data from `t-1` and earlier**
* We prevent including the value from day `t` in its own rolling calculation by applying `.shift(1)` to the data *before* the `.rolling()` operation.

In [17]:
# --- 4.3.1: Lag Features ---
# Define the features to lag and the time periods for the lookback.
# We choose key dynamic variables that characterize the daily weather system.
features_to_lag = [
    "temp",
    "humidity",
    "solarradiation",
    "precip",
    "windspeed",
    "cloudcover",
    "dew",
    "sealevelpressure",
    "windgust",
]
lag_periods = [1, 2, 3, 7, 14, 28]

for feature in features_to_lag:
    for lag in lag_periods:
        df_engineered[f"{feature}_lag_{lag}"] = df_engineered[feature].shift(lag)

# --- 4.3.2: Rolling Window Features ---
# Define features and windows for rolling statistics.
features_for_rolling = [
    "temp",
    "humidity",
    "windspeed",
    "solarradiation",
    "dew",
    "cloudcover",  # Precip has its own logic
]
window_sizes = [7, 14, 28]
rolling_stats = ["mean", "std"]

for feature in features_for_rolling:
    # Shift the series by 1 to prevent data leakage.
    series_shifted = df_engineered[feature].shift(1)
    for window in window_sizes:
        for stat in rolling_stats:
            col_name = f"{feature}_roll_{window}d_{stat}"
            df_engineered[col_name] = series_shifted.rolling(window=window, min_periods=1).agg(stat)

# Specific logic for precipitation: sum is more meaningful than mean.
precip_shifted = df_engineered["precip"].shift(1)
for window in window_sizes:
    df_engineered[f"precip_roll_{window}d_sum"] = precip_shifted.rolling(window=window, min_periods=1).sum()


shape_before_fe = df_processed.shape[1]
shape_after_fe = df_engineered.shape[1]
num_new_features = shape_after_fe - shape_before_fe - len(temporal_features_added) - len(domain_features_added)

console.print(
    "[bold]Lag and rolling features created.[/bold]\n",
    f" - Number of new lag/rolling features: [green]{num_new_features}[/green]\n",
    f" - DataFrame shape is now: [green]{df_engineered.shape}[/green]",
)


Lag and rolling features created.
  - Number of new lag/rolling features: 93
  - DataFrame shape is now: (3927, 132)

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-categorical" style="display: none;">Q4.4. Categorical Feature Encoding</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-categorical-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.4. Categorical Feature Encoding</h2>

The dataset contains one structured categorical feature, `preciptype`, which currently holds the values `'rain'` and `'none'`. We must convert it into a numerical format so the model can understand it.

*   **Chosen method: One-hot Encoding**
    *   We will use One-hot Encoding to convert the single `preciptype` column into two new binary (0/1) columns: `preciptype_rain` and `preciptype_none`.
*   **Justification:**
    *   We didn't choose Dummy Encoding (`k-1` columns) because it preserves the explicit interpretability of each category.
    *   While creating `k` columns can introduce perfect multicollinearity (since one column can be perfectly predicted by the others), this is not a concern for the tree-based models. For linear models, the built-in regularization will effectively handle this redundancy. Therefore, prioritizing clarity and interpretability is the better choice.

In [18]:
# Perform One-Hot Encoding on the 'preciptype' column
df_engineered = pd.get_dummies(
    df_engineered,
    columns=["preciptype"],
    prefix="preciptype",
    dtype=int,
)


encoded_cols = [col for col in df_engineered.columns if "preciptype_" in col]
console.print(
    "[bold]Original 'preciptype' column removed.[/bold]\n",
    f" - New binary columns added: `{encoded_cols}`\n",
    f" - DataFrame shape is now: [green]{df_engineered.shape}[/green]",
)

console.print("\n[bold]Sample of new encoded features:[/bold]")
display(df_engineered[encoded_cols].head())


Original 'preciptype' column removed.
  - New binary columns added: `['preciptype_none', 'preciptype_rain']`
  - DataFrame shape is now: (3927, 133)

Sample of new encoded features:

,preciptype_none,preciptype_rain
datetime,,
2015-01-01,1,0
2015-01-02,1,0
2015-01-03,0,1
2015-01-04,1,0
2015-01-05,0,1


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-text" style="display: none;">Q4.5. Unstructured Text Feature Engineering</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-text-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.5. Unstructured Text Feature Engineering</h2>

* The dataset contains a `description` column with human-readable weather summaries (e.g., "Partly cloudy throughout the day with rain in the morning."). While our numerical features capture quantitative data, this text may contain valuable qualitative nuances.
* To leverage this information, we will employ a standard and robust Natural Language Processing (NLP) pipeline to convert the text into a set of numerical features.
*   **Chosen method: TF-IDF + Truncated SVD**
    1.  **TF-IDF (Term Frequency-Inverse Document Frequency):** converts the text into a high-dimensional matrix, where each value represents the importance of a word to a specific day's description.
    2.  **Truncated SVD (Singular Value Decomposition):** The TF-IDF matrix is very sparse and has too many dimensions (one for each unique word) $\rightarrow$ Truncated SVD will reduce this matrix into a small number of dense components (5 in our case). In other words, we can understand it as capturing the core "topics" or "concepts" within the weather descriptions.


In [19]:
text_data = df_raw[["description"]].copy()
df_engineered = df_engineered.join(text_data)

# --- Define the NLP processing pipeline ---
n_svd_components = 7  # The `description` field has 26 distinct features, mostly repetitive, so 7 is robust for our case
nlp_pipeline = Pipeline([
    (
        "tfidf",
        TfidfVectorizer(
            stop_words="english",
            ngram_range=(1, 2),  # Capture both single words and two-word phrases
            max_features=1000,  # Limit vocabulary to the top 1000 terms
        ),
    ),
    ("svd", TruncatedSVD(n_components=n_svd_components, random_state=config.GLOBAL_RANDOM_SEED)),
])

text_features = nlp_pipeline.fit_transform(df_engineered["description"])

text_feature_names = [f"text_svd_{i}" for i in range(n_svd_components)]
df_text_features = pd.DataFrame(text_features, index=df_engineered.index, columns=text_feature_names)

df_engineered = pd.concat([df_engineered, df_text_features], axis=1)

df_engineered.drop(columns=["description"], inplace=True)


console.print(
    f" - Number of new text-based features (SVD components): [green]{n_svd_components}[/green]\n",
    f" - DataFrame shape is now: [green]{df_engineered.shape}[/green]",
    sep="",
)

console.print("\n[bold]Sample of new text features:[/bold]")
display(df_engineered[text_feature_names].head())


- Number of new text-based features (SVD components): 7
 - DataFrame shape is now: (3927, 140)

Sample of new text features:

,text_svd_0,text_svd_1,text_svd_2,text_svd_3,text_svd_4,text_svd_5,text_svd_6
datetime,,,,,,,
2015-01-01,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-02,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-03,0.555798,-0.049113,0.585367,0.280771,-0.240929,0.061356,0.448770
2015-01-04,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-05,0.407113,-0.081170,0.087575,0.089282,-0.493769,-0.186068,-0.152486


<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="fe-final-verification" style="display: none;">Q4.6. Final Verification & NaN Handling</h2>
<!-- VISIBLE H2 -->
<h2 id="fe-final-verification-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q4.6. Final Verification & NaN Handling</h2>

Now we verify the structure and acknowledge the presence of `NaN` values. These `NaN`s are an expected and correct outcome of the lag and rolling window operations. The initial rows of the DataFrame lack sufficient historical data for these calculations (e.g., since the first day has no "day before" to lag from).

The number of rows containing these `NaN`s is determined by our longest lookback period. These rows will be dropped in the next section, **after** we create our target variables. This ensures a perfect, leak-free alignment between our features (`X`) and the values we aim to predict (`y`).

In [20]:
max_lag = max(lag_periods)
max_rolling_window = max(window_sizes)
max_lookback = max(max_lag, max_rolling_window)

console.print(
    f"Longest lag period: [cyan]{max_lag}[/cyan] days\n",
    f"Longest rolling window: [cyan]{max_rolling_window}[/cyan] days\n",
    f"==> Maximum lookback period: [bold yellow]{max_lookback}[/bold yellow] days\n",
    f"\nThis means the first {max_lookback} rows will contain NaN values and will be dropped during the modeling prep stage.\n",
    f"[bold]Final Shape of DataFrame:[/bold] [green]{df_engineered.shape}[/green]",
    sep="",
)

console.print("[bold]First 5 Rows (showing initial NaNs):[/bold]")
display(df_engineered.head())


Longest lag period: 28 days
Longest rolling window: 28 days
==> Maximum lookback period: 28 days

This means the first 28 rows will contain NaN values and will be dropped during the modeling prep stage.
Final Shape of DataFrame: (3927, 140)

First 5 Rows (showing initial NaNs):

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,...,precip_roll_28d_sum,preciptype_none,preciptype_rain,text_svd_0,text_svd_1,text_svd_2,text_svd_3,text_svd_4,text_svd_5,text_svd_6
datetime,,,,,,,,,,,,,,,,,,,,,
2015-01-01,31.0,23.0,26.6,31.4,23.0,26.8,17.9,60.3,0.0,0,...,NaN,1,0,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-02,30.0,20.0,25.0,30.4,20.0,25.1,15.8,57.2,0.0,0,...,0.0,1,0,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-03,32.0,23.0,26.8,33.5,23.0,27.4,19.0,63.7,0.2,100,...,0.0,0,1,0.555798,-0.049113,0.585367,0.280771,-0.240929,0.061356,0.448770
2015-01-04,32.0,24.0,27.1,34.8,24.0,28.3,21.3,71.7,0.0,0,...,0.2,1,0,0.879053,-0.176581,-0.320034,-0.237530,-0.112032,-0.016034,0.155117
2015-01-05,30.6,24.9,26.7,33.2,24.9,27.7,22.1,76.4,6.0,100,...,0.2,0,1,0.407113,-0.081170,0.087575,0.089282,-0.493769,-0.186068,-0.152486


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="modeling" style="display: none;">Q5. Modeling Strategy & Evaluation</h1>
<!-- VISIBLE H1 -->
<h1 id="modeling-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">Q5. Modeling Strategy & Evaluation</h1>

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="modeling-foundational-setup" style="display: none;">Q5.1. Forecasting Strategies, Target Definition & Data Splitting</h2>
<!-- VISIBLE H2 -->
<h2 id="modeling-foundational-setup-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.1. Forecasting Strategies, Target Definition & Data Splitting</h2>

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h3 id="modeling-strategy" style="display: none;">Q5.1.1. Defining the Forecasting Strategy</h3>
<!-- VISIBLE H3 -->
<h3 id="modeling-strategy-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.1.1. Defining the Forecasting Strategy</h3>

### Forecasting Architectures

**1. Recursive strategy:**
*   **Method:** A single model is trained to predict only one step ahead (`t+1`). To forecast `t+2`, the model's own prediction for `t+1` is fed back into its input features. This process is repeated for each subsequent step.
*   **Drawback:** This approach is highly susceptible to **error accumulation**. Any small error in the `t+1` forecast becomes part of the input for the `t+2` forecast, and these errors compound, often leading to a rapid degradation in performance for longer horizons.

**2. Multi-output strategy:**
*   **Method:** A single, more complex model is trained to predict all five forecast horizons (`t+1` through `t+5`) simultaneously in one forward pass.
*   **Drawback:** This model is a "generalist." While computationally efficient, it often struggles to optimize for any single horizon, resulting in performance that is good on average but rarely the best for any specific day.

**3. Direct strategy (We choose this):**
*   **Method:** We train a separate, independent, specialist model for each forecast horizon. The model for `t+3`, for example, is trained exclusively on data pairs of `(Features at time t, Target at time t+3)`.
*   **Justification:** This is the most robust and potentially highest-performing strategy for this problem. It avoids error accumulation and allows each model to become an expert at its specific task, learning the unique feature relationships that are most important for that particular forecast distance. While it requires training multiple models, the gain in accuracy and reliability is a worthwhile trade-off, especially since the amount of training data for the daily dataset is relatively small, and  only have to predict 5 models (5 horizons).

### What to Predict?

**1. Predicting the *direct value* (We choose this):** The most straightforward and robust approach is to predict the final temperature value directly. The previous feature engineering step (lags, rolling windows, cyclical features) has already provided the model with most necessary information about trends, seasonality, and recent changes, allowing the model to learn the complete mapping from features to the final target value end-to-end.

**2. Predicting the *delta*:** We could predict the *change* in temperature from day `t` to `t+h`. The final forecast is then `Temp_t + Predicted_Delta`. This can be effective if the series is strongly trending, but it can be unstable. For temperature, especially for a relatively "stable" area like Ho Chi Minh city, the deltas wouldn't be clear, doesn't have strong strends (it has some, as noted in EDA above, but not clear enough of a signal), which makes it hard for the model to reliably predict the change in temperature from day to day with great accuracy.

**3. Predicting the *residual*:** After de-trending and de-seasonalizing (as in our STL decomposition), we could model the remaining "noise" or residual. It's more common technique in classical time-series analysis but often more complex to implement in ML effectively, and same with predicting the delta approach above, the residual don't have any strong, clear trends or big enough of a difference for the model to reliably learn the pattern and predict.

$\implies$ We will implement a **direct forecasting strategy** where we train **five specialist models**, each tasked with predicting the **absolute temperature value** for its assigned horizon (`t+1` through `t+5`).

<!-- HIDDEN H3 FOR OUTLINE VIEW -->
<h3 id="modeling-target-creation" style="display: none;">Q5.1.2. Target Variable Creation & Final Alignment</h3>
<!-- VISIBLE H3 -->
<h3 id="modeling-target-creation-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.1.2. Target Variable Creation & Final Alignment</h3>

Since we're using the direct forecasting strategy, we need to create the five distinct target variables, done by shifting the `temp` column backwards in time. For example, the target for day `t`'s features will be the actual temperature from day `t+1`, `t+2`, and so on.

This will introduce `NaN` values at the end of the DataFrame (as there is no future data for the last few days) and at the beginning (from the lag/rolling features). We will then need to drop all rows containing any `NaN` values, ensuring every row in the final DataFrame is complete and correct.

In [21]:
df_model_ready = df_engineered.copy()

# --- 1. Create the Multi-Horizon Target Variables ---
for h in config.HORIZONS:
    df_model_ready[f"target_temp_t+{h}"] = df_model_ready["temp"].shift(-h)

# --- 2. Final Alignment via NaN Removal ---
shape_before_align = df_model_ready.shape
df_model_ready.dropna(inplace=True)
shape_after_align = df_model_ready.shape

rows_dropped = shape_before_align[0] - shape_after_align[0]

console.print(
    f"Shape before final alignment: [green]{shape_before_align}[/green]\n",
    f"Shape after final alignment:  [green]{shape_after_align}[/green]\n",
    f"Rows dropped due to lookback/lookahead: [red]{rows_dropped}[/red]",
    sep="",
)


Shape before final alignment: (3927, 145)
Shape after final alignment:  (3894, 145)
Rows dropped due to lookback/lookahead: 33

<!-- HIDDEN H3 FOR OUTLINE VIEW -->
<h3 id="modeling-final-split" style="display: none;">Q5.1.3. The Final Train-Test Split</h3>
<!-- VISIBLE H3 -->
<h3 id="modeling-final-split-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.1.3. The Final Train-Test Split</h3>

*   **Training Set (`_train`):** This data (the first 85%) is used for all model training, cross-validation, and hyperparameter tuning.
*   **Test Set (`_test`):** This data (the final 15%) is held out and completely untouched until the very end of the project. The set simulates the future, unseen datawhich the top model model will be evaluated on this set *once* to produce its final, unbiased performance score.

We adopt this **Train/Test with Cross-Validation** approach over a fixed Train/Validation/Test split:
* A fixed validation set can be small and idiosyncratic, leading a model to overfit to its specific patterns
* By using `TimeSeriesSplit` cross-validation on the larger training set, we evaluate our models across multiple, diverse time periods, leading to a much more robust and reliable estimate of their true generalization performance.
* After evaluating the models to choose the most suitable one, we will train that model once on the entire training set, providing it with more recent data (closer to the test set). This is even more important because weather/climate is non-stationary, we may have data drift and/or concept drift, so the model may perform differently (usually worse) as its training data and test data grows further apart. More on that in Q7.

In [22]:
# --- 1. Define Feature and Target Columns ---
target_cols = [f"target_temp_t+{h}" for h in config.HORIZONS]

# We must drop the original 'temp' column as is now a source of data leakage (because it's
# from the same day as the target values we are trying to predict)
features_to_drop = target_cols + ["temp"]

# All remaining columns are predictors.
feature_cols = [col for col in df_model_ready.columns if col not in features_to_drop]

X = df_model_ready[feature_cols]
y = df_model_ready[target_cols]

# --- 2. Perform the Chronological Split ---
n_total = len(X)
n_test = int(n_total * config.TEST_SIZE)
n_train = n_total - n_test

X_train_full = X.iloc[:n_train]
y_train = y.iloc[:n_train]

X_test_full = X.iloc[n_train:]
y_test = y.iloc[n_train:]


console.print(
    f" - Total samples:     {n_total}\n",
    f" - Training set size: {n_train} ({1 - config.TEST_SIZE:.0%})\n",
    f" - Test set size:     {n_test} ({config.TEST_SIZE:.0%})",
    sep="",
)

console.rule("[bold]Time Range of the sets:[/bold]", align="center")
console.print(
    f"Training Set Time Range:  {X_train_full.index.min().date()} to {X_train_full.index.max().date()}\n",
    f"Test Set Time Range:      {X_test_full.index.min().date()} to {X_test_full.index.max().date()}",
    sep="",
)


- Total samples:     3894
 - Training set size: 3310 (85%)
 - Test set size:     584 (15%)

───────────────────────────────────────────── Time Range of the sets: ─────────────────────────────────────────────

Training Set Time Range:  2015-01-29 to 2024-02-20
Test Set Time Range:      2024-02-21 to 2025-09-26

<!-- HIDDEN H3 FOR OUTLINE VIEW -->
<h3 id="modeling-feature-subsets" style="display: none;">Q5.1.4. Creating Specialized Feature Subsets</h3>
<!-- VISIBLE H3 -->
<h3 id="modeling-feature-subsets-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.1.4. Creating Specialized Feature Subsets</h3>

As established in our EDA, severe multicollinearity poses a significant challenge for linear models. To address this, we now create two distinct, purpose-built feature sets for our upcoming model bake-off:

1.  **`X_train_tree` / `X_test_tree`:** A comprehensive (full) feature set containing all engineered predictors. This set is designed for tree-based models (like LightGBM, XGBoost) and other non-linear architectures which are inherently robust to multicollinearity.

2.  **`X_train_linear` / `X_test_linear`:** A carefully curated, parsimonious feature set for our linear models (`RidgeCV`, `ElasticNetCV`, etc.). This subset will be created with data-driven methodology that penalizes feature redundancy.

**We choose: `MultiTaskLassoCV`**

*   **What it is:** Lasso (Least Absolute Shrinkage and Selection Operator) is a form of linear regression that includes an $L_1$ penalty term. This penalty forces the coefficients of less important or redundant features to shrink to exactly zero, effectively performing automated feature selection.
*   **Why `MultiTask`?** Standard Lasso works on a single target. Since we have five forecast horizons, `MultiTaskLassoCV` jointly fits models for all five targets simultaneously. It applies a group penalty that encourages the selection of a common set of features that are useful *across all horizons*. This is ideal for finding a single, robust feature set for our entire direct forecasting system. We could use `LassoCV` for each of the horizon, but that increases uncessary complexity, reduce interpretability, all without much (if any) meaningful gains.
*   **Why `CV`?** The `CV` (Cross-Validation) component automatically finds the optimal strength of the regularization penalty (`alpha`) using our `TimeSeriesSplit` cross-validation strategy, ensuring the feature selection process is itself robust and leak-free.

In essence, by fitting `MultiTaskLassoCV` on the full training set, we are asking the data itself to identify the most potent and non-redundant subset of predictors, creating a lean, powerful feature set perfectly tailored for our linear models.

In [23]:
X_train_tree = X_train_full
X_test_tree = X_test_full

# --- Use MultiTaskLassoCV to select a low-collinearity subset for linear models ---
# Fit only on the training data
lasso_pipeline = Pipeline([
    ("scaler", RobustScaler()),
    (
        "lasso",
        MultiTaskLassoCV(
            cv=TimeSeriesSplit(n_splits=5),
            random_state=config.GLOBAL_RANDOM_SEED,
            n_jobs=-1,
        ),
    ),
])

lasso_pipeline.fit(X_train_full, y_train)

# Identify features with non-zero coefficients.
coefficients = lasso_pipeline.named_steps["lasso"].coef_
selected_features_mask = np.abs(coefficients).sum(axis=0) > 0
linear_feature_cols = X_train_full.columns[selected_features_mask].tolist()

X_train_linear = X_train_full[linear_feature_cols]
X_test_linear = X_test_full[linear_feature_cols]


console.print(
    f" - Tree-based model feature count: [cyan]{X_train_tree.shape[1]}[/cyan]\n",
    f" - Linear model feature count:     [cyan]{X_train_linear.shape[1]}[/cyan] (selected by LASSO)",
    sep="",
)


- Tree-based model feature count: 139
 - Linear model feature count:     41 (selected by LASSO)

In [24]:
ah.display_lasso_summary_table(
    all_features=X_train_tree.columns.tolist(),
    selected_features=X_train_linear.columns.tolist(),
)

# fig_sunburst_final = ah.plot_lasso_summary_sunburst(
#     all_features=X_train_tree.columns.tolist(),
#     selected_features=X_train_linear.columns.tolist(),
# )
# fig_sunburst_final.show()


           LASSO Feature Selection Summary           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Feature Category         ┃ Kept ┃ Dropped ┃ Total ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ Raw Predictor            │  13  │    9    │  22   │
│ Temporal (Cyclical)      │  4   │    4    │   8   │
│ Text (SVD)               │  3   │    4    │   7   │
│ Lag (Humidity)           │  1   │    5    │   6   │
│ Lag (Dew)                │  2   │    4    │   6   │
│ Lag (Cloudcover)         │  1   │    5    │   6   │
│ Rolling (Dew)            │  0   │    6    │   6   │
│ Rolling (Cloudcover)     │  2   │    4    │   6   │
│ Lag (Precip)             │  6   │    0    │   6   │
│ Lag (Sealevelpressure)   │  2   │    4    │   6   │
│ Lag (Windgust)           │  0   │    6    │   6   │
│ Lag (Windspeed)          │  0   │    6    │   6   │
│ Lag (Temp)               │  3   │    3    │   6   │
│ Lag (Solarradiation)     │  0   │    6    │   6   │
│ Rolling (Windspeed)      │  1   │    5    │   6   │
│ Rolling (Temp)           │  2   │    4    │   6   │
│ Rolling (Humidity)       │  0   │    6    │   6   │
│ Rolling (Solarradiation) │  1   │    5    │   6   │
│ Temporal (Basic)         │  0   │    4    │   4   │
│ Domain-Specific          │  0   │    3    │   3   │
│ Rolling (Precip)         │  0   │    3    │   3   │
│ Categorical              │  0   │    2    │   2   │
└──────────────────────────┴──────┴─────────┴───────┘

LASSO provided insights into which features are most valuable for a *linear* forecasting model.

*   **Aggressive pruning of redundancy:** LASSO reduced the feature set by 70%, from 139 down to 41. It heavily pruned the highly collinear **Lag** and **Rolling Window** features, which made up the bulk of the original set.

*   **Preference for recent & long-term history:**
    *   For the primary `temp` variable, LASSO kept the most recent history (`temp_lag_1`, `_2`, `_3`), confirming that yesterday's temperature is a powerful predictor.
    *   Interestingly, for other atmospheric variables like `dew` and `cloudcover`, it discarded recent lags but kept the **long-term lags (14 and 28 days)**, *consistently*. This suggests that for a linear model, the broader climate context from 2-4 weeks ago is more valuable than the noisy data from 1-2 days ago.

*   **Value of raw predictors:** LASSO retained a core set of 13 raw predictors, including fundamental drivers like `solarradiation` and `cloudcover`, after deeming many of their derivative lag/rolling features redundant.

*   **Rejection of complex/derived Features:**
    *   The model dropped **all domain-specific features** (`daylight_hours`, `temp_range`, etc.) and **all categorical flags** (`preciptype_none/rain`).
    *   Perhaps, for a linear model, the information contained in these features was likely already captured more effectively by the combination of raw predictors and selected lags. For example, the information in `temp_range` is already present in the raw `tempmax` and `tempmin` features, which LASSO chose to keep.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="evaluation-framework" style="display: none;">Q5.2. Evaluation Framework</h2>
<!-- VISIBLE H2 -->
<h2 id="evaluation-framework-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.2. Evaluation Framework</h2>

<!-- HIDDEN H3 FOR OUTLINE VIEW -->
<h3 id="evaluation-metrics" style="display: none;">Q5.2.1. Evaluation Metrics</h3>
<!-- VISIBLE H3 -->
<h3 id="evaluation-metrics-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.2.1. Evaluation Metrics</h3>

Before training any models, we must define how we will measure "good." A single metric is rarely sufficient; a holistic view requires assessing performance from multiple perspectives. The following table defines the standard regression metrics we will consider.

| METRIC         | FORMULA                                                                                             | CORE QUESTION                                                                    | INTEPRETATIONS                                                                                                                  |
| :-------------- | :---------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------- |
| **R-squared ($R^2$)** | $1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$                        | "How much of the temperature's variance is explained by our model?"              | A scale-free score from $(-\infty, 1]$. A score of 0.7 means our model explains 70% of the variance. **Primary metric for model comparison.** |
| **Adjusted $R^2$** | $1 - \frac{(1-R^2)(n-1)}{n-p-1}$                                                                       | "What is the $R^2$, after penalizing for adding more features?"                | Useful for explanatory linear modeling to avoid overfitting by adding useless predictors. **Not used in our final reports.**                    |
| **RMSE**       | $\sqrt{\frac{1}{n} \sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$                                                  | "What is the typical magnitude of our error, penalizing larger errors more?"     | Also in degrees Celsius. Because errors are squared, RMSE is more sensitive to large forecast misses than MAE. A key indicator of error size. |
| **MAE**        | $\frac{1}{n} \sum_{i=1}^{n} \|y_i - \hat{y}_i\|$                                                            | "On average, what is the absolute error of our forecast in degrees?"             | Highly interpretable. An MAE of 0.5 means our forecasts are off by an average of 0.5°C. Robust to outliers.                                    |
| **MAPE**       | $\frac{100}{n} \sum_{i=1}^{n} \|\frac{y_i - \hat{y}_i}{y_i}\|$                                                | "On average, what is the percentage error of our forecast?"                      | A relative error metric. A MAPE of 2% means our forecasts are off by an average of 2% of the actual temperature.                               |



Based on the definitions above, we will adopt the following reporting hierarchy:
1.  **Primary metric for selection:** **R-squared ($R^2$)** will be the primary metric used in our bake-off leaderboards to compare the relative performance of different model architectures.
2.  **Key error metrics for interpretation:** **RMSE** and **MAE** will be reported for our finalist and champion models. They provide a direct, interpretable measure of the forecast error in degrees Celsius.
3.  **Secondary relative metric:** **MAPE** will be reported for the final champion model to provide a sense of percentage error.
4.  **Excluded metric:** We will **not** report **Adjusted $R^2$**. For our predictive task, the use of a robust cross-validation strategy on held-out data (`TimeSeriesSplit`) is a more direct and applicable method for penalizing model complexity and assessing true generalization performance.

<!-- HIDDEN H3 FOR OUTLINE VIEW -->
<h3 id="cv-strategy" style="display: none;">Q5.2.2. Cross-Validation Strategy</h3>
<!-- VISIBLE H3 -->
<h3 id="cv-strategy-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 16px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.2.2. Cross-Validation Strategy</h3>

To reliably estimate a model's performance on unseen data, we must use a cross-validation strategy that respects the temporal nature of our data.

*   **Invalid method: standard K-Fold CV**
    *   Standard K-Fold randomly shuffles and splits the data into `k` folds.
    *   For time-series data, this is a catastrophic error. It allows the model to train on data from the future (e.g., 2023) to make predictions on data from the past (e.g., 2021), which is **data leakage**, leading to wildly optimistic and completely invalid performance estimates.

*   **Chosen method: `TimeSeriesSplit`**
    *   `TimeSeriesSplit` creates a series of expanding or sliding window splits. In each fold, the training set always consists of data that occurred *before* the validation set.

    *   **How it Works (for 5 splits):** Split the train set into 6 blocks
        *   **Fold 1:** Train on `[Block 1]`, validate on `[Block 2]`
        *   **Fold 2:** Train on `[Block 1, Block 2]`, validate on `[Block 3]`
        *   **Fold 3:** Train on `[Block 1, 2, 3]`, validate on `[Block 4]`
        *   ...and so on.

$\implies$ This process perfectly simulates a real-world production scenario where a model is periodically retrained on all available historical data to make forecasts for the immediate future. The average score across these folds provides a robust, realistic, and leak-free estimate of our model's generalization ability. All model selection decisions in this notebook will be based on performance using this `TimeSeriesSplit` strategy.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="baseline-model" style="display: none;">Q5.3. The Baseline Model: Establishing the Performance Floor</h2>
<!-- VISIBLE H2 -->
<h2 id="baseline-model-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.3. The Baseline Model: Establishing the Performance Floor</h2>

A baseline provides the "performance floor"—the minimum score that any sophisticated model must convincingly beat to be considered useful. Without a baseline, a model's performance score lacks context.

Several options for a baseline exist (for the current daily dataset):

*   **Historical average:** Predict every day with the overall average temperature from the training set. This is the simplest possible baseline, but it is too naive for our data as it completely ignores the powerful seasonal cycle we identified.

*   **Seasonal naive:** Predict the temperature for a given day using the temperature from the same day one year prior (e.g., $\hat{y}_t = y_{t-365}$). This is a stronger baseline that accounts for seasonality.

*   **Persistence model (We choose this):** This model operates on a simple, powerful heuristic: "the best guess for the near future is the recent past." It predicts the temperature for a future day `t+h` using the temperature from day `t`.
    *   **Formula:** $\hat{y}_{t+h} = y_t$
    *   **Justification:** We choose the persistence model as our primary baseline because it represents the most fundamental temporal relationship in the data. For short horizons like `t+1`, it can be surprisingly difficult to beat. Any model we build must first demonstrate that it can leverage our rich feature set to understand the weather dynamics *better* than this simple persistence rule. It provides the true, lowest bar for a time-aware forecast.

In [25]:
# baseline_cv_scores = eval.evaluate_persistence_model(
#     df_model_ready=df_model_ready,
#     X_train=X_train_full,
#     y_train_all_horizons=y_train,
#     horizons=config.HORIZONS,
# )
# eval.display_baseline_results(baseline_cv_scores)


*   **Decent short-term performance:** A mean CV R² of **0.5455 for the `t+1` horizon** is decent for a naive model. It confirms that in Ho Chi Minh's stable weather system, today's temperature is a strong predictor of tomorrow's. Our `t+1` models must convincingly beat this high bar.

*   **Rapid performance decay:** As expected, the model's performance collapses as the forecast horizon increases, becoming worse than a simple historical average by `t+5` (negative R² score). This highlights the core challenge of our project: predicting temperature several days in advance, where simple persistence fails completely.

*   **The importance of a robust CV score:**
    *   The average CV score (e.g., 0.5455 for `t+1`) is a conservative but realistic estimate of the model's performance over time.
    *   It is expected to be lower than a one-off score on the final test set (which for `t+1` is ~0.685) because the `TimeSeriesSplit` forces the model to make predictions across various time periods, including older data where patterns might be different. A model that performs well across all CV folds is more robust and reliable than one that only performs well on the most recent data.

$\implies$ **Goal:** Our models must demonstrate their value by not only outperforming this baseline on average, but especially by providing a significant performance lift on the challenging **`t+3` to `t+5` horizons**, where the naive model provides no value.

<!-- HIDDEN H2 FOR OUTLINE VIEW -->
<h2 id="bakeoff" style="display: none;">Q5.4. Model Bake-Off</h2>
<!-- VISIBLE H2 -->
<h2 id="bakeoff-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', 'sans-serif'; color: #38545f; font-size: 18px; font-weight: 500; background-color: #f9fbfb; border-top: 4px solid #0c75ab; border-radius: 2px; border-bottom: 1px solid #D9D9D9; padding: 10px 0px 10px 15px; margin-top: 15px;">Q5.4. Model Bake-Off</h2>

We will now systematically evaluate a diverse range of model architectures to identify the most promising candidate for our champion model. This process is conducted with the following protocol:

*   **Comprehensive saves:** We use the `run_bakeoff` utility function to generate and save detailed result files (`.pkl`). These files contain not just the final scores, but the scores and predictions from *every fold*, allowing for deep diagnostic analysis.
*   **Presentation reporting:** We then load those pre-computed artifacts to generate summary tables and visualizations. The `config.TRAIN_NEW_MODELS` flag allows us to optionally re-run the training process if needed, but load the pre-computed ones by default to save time.

We will evaluate four main families of models:
1.  **Standard ML models:** Linear and tree-based models that serve as strong, interpretable baselines.
2.  **Advanced time-series models:** Architectures like SARIMAX and Prophet that are specifically designed for time-series data.
3.  **Deep learning models**: Models like LSTM, CNN, Seq2Seq, with attention strategy may capture the pattern better, especially with longer horizons.
4.  **Ensemble models:** Hybrid models that combine the predictions of multiple base models to improve robustness and accuracy.

In [26]:
# sandbox_notebook.ipynb - Bake-Off Script

# --- 1. Imports & Setup ---
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Assuming all utility functions and the ChampionForecastStack class are in utils/
import utils.training as train

# Create a directory for experiment artifacts if it doesn't exist
os.makedirs("experiments", exist_ok=True)

# --- 2. Load Data (Assuming this is already done in your sandbox) ---
# You should have:
# X_train_linear, X_train_tree
# y_train
# config (with HORIZONS and GLOBAL_RANDOM_SEED)

# --- 3. Define Models for Bake-Off ---
# Naming convention is key for clarity in results.
linear_models = {
    "LinearRegression": LinearRegression(n_jobs=-1),
    "RidgeCV": RidgeCV(cv=TimeSeriesSplit(n_splits=5)),
    "ElasticNetCV": ElasticNetCV(
        cv=TimeSeriesSplit(n_splits=5),
        n_jobs=-1,
        random_state=config.GLOBAL_RANDOM_SEED,
    ),
}

tree_models = {
    "LGBMRegressor": LGBMRegressor(
        random_state=config.GLOBAL_RANDOM_SEED,
        n_jobs=-1,
        verbosity=-1,
    ),
    "XGBRegressor": XGBRegressor(
        random_state=config.GLOBAL_RANDOM_SEED,
        n_jobs=-1,
    ),
    "CatBoostRegressor": CatBoostRegressor(
        random_state=config.GLOBAL_RANDOM_SEED,
        verbose=0,
    ),
}

# --- 4. Define CV Splitter ---
tscv_splitter = TimeSeriesSplit(n_splits=5)


# **> Run from above to here <**

In [ ]:
# Empty

### **Bake-off for the single models and ensembles**

In [ ]:
# --- 5. Run Bake-Offs and Save Artifacts ---

# Run for Linear Models
print("--- Running Bake-Off for Linear Models ---")
linear_results = train.run_bakeoff(
    models_to_run=linear_models,
    X_train=X_train_linear,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(linear_results, "experiments/bakeoff_results_linear.pkl")
print("Linear model results saved to 'experiments/bakeoff_results_linear.pkl'")

# Run for Tree-Based Models
print("\n--- Running Bake-Off for Tree-Based Models ---")
tree_results = train.run_bakeoff(
    models_to_run=tree_models,
    X_train=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(tree_results, "experiments/bakeoff_results_tree.pkl")
print("Tree-based model results saved to 'experiments/bakeoff_results_tree.pkl'")


In [ ]:
# sandbox_notebook.ipynb - Advanced TS Bake-Off Script

# --- 1. Imports for Advanced Models ---
# Make sure you have these installed: pip install statsmodels prophet
import statsmodels.api as sm
from prophet import Prophet
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress Prophet warnings

# (Existing imports like os, joblib, pandas, etc., are assumed to be present)
# import utils.training as train (if in a new session)

# --- 2. Define Custom Wrapper for Prophet ---
# Prophet has a different API, so we wrap it in a scikit-learn compatible class.
from sklearn.base import BaseEstimator, RegressorMixin


def _calculate_mape(y_true, y_pred):
    """Helper to calculate MAPE safely."""
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-8))) * 100


class ProphetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, seasonality_mode="additive"):
        self.seasonality_mode = seasonality_mode
        self.model = None
        self.regressor_names = []

    def fit(self, X, y):
        # --- FIX: Add robust imputation as a safeguard ---
        X_imputed = X.ffill().bfill()

        # --- FIX: Correct DataFrame construction to prevent index misalignment ---
        # Start with the features and add 'ds' and 'y' columns.
        df_prophet_input = X_imputed.copy()
        df_prophet_input["ds"] = df_prophet_input.index
        df_prophet_input["y"] = y.values  # Use .values to ignore index and align by position

        self.regressor_names = [col for col in X.columns]
        self.model = Prophet(seasonality_mode=self.seasonality_mode)
        for regressor in self.regressor_names:
            self.model.add_regressor(regressor)

        # Suppress verbose output from Prophet
        with open(os.devnull, "w") as devnull:
            import sys

            old_stdout = sys.stdout
            sys.stdout = devnull
            try:
                # Use the correctly constructed DataFrame
                self.model.fit(df_prophet_input)
            finally:
                sys.stdout = old_stdout
        return self

    def predict(self, X):
        # --- FIX: Ensure prediction data is also imputed ---
        X_imputed = X.ffill().bfill()

        # --- FIX: Correct DataFrame construction for prediction ---
        df_future = X_imputed.copy()
        df_future["ds"] = df_future.index

        forecast = self.model.predict(df_future)
        return forecast["yhat"].values


# --- 3. Define Advanced Models ---
# Note: SARIMAX is extremely slow and might not be practical for a full CV bake-off
# on a large feature set. We include it for completeness but be aware of runtime.
# For SARIMAX, we will use a reduced feature set for tractability.

# A smaller, highly relevant feature set for SARIMAX
sarimax_features = ["sin_day_of_year", "cos_day_of_year", "temp_lag_1", "temp_roll_7d_mean", "solarradiation_lag_1"]
# X_train_sarimax = X_train_tree[sarimax_features]
X_train_sarimax = X_train_linear

advanced_ts_models = {
    "Prophet": ProphetWrapper(),
    # SARIMAX(order=(p,d,q), seasonal_order=(P,D,Q,s))
    # We use a common simple order. Finding the best order is a project in itself.
    # We use the full feature set as exogenous variables ('exog').
    "SARIMAX": sm.tsa.SARIMAX(
        endog=y_train["target_temp_t+1"],  # Will be replaced in loop
        exog=X_train_sarimax,  # Will be replaced in loop
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 7),
    ),
}

# --- 4. Special Bake-Off Loop for Advanced Models ---
# We need a custom loop because the statsmodels API is different.
advanced_ts_results = {}
tscv_splitter = TimeSeriesSplit(n_splits=5)

for model_name, model_obj in advanced_ts_models.items():
    print(f"\n--- Running Bake-Off for {model_name} ---")
    model_results = {}

    for h in tqdm(config.HORIZONS, desc=f"Training {model_name}"):
        target_col = f"target_temp_t+{h}"
        y_train_horizon = y_train[target_col]

        fold_scores = {"r2": [], "rmse": [], "mae": [], "mape": []}

        for train_idx, val_idx in tscv_splitter.split(X_train_tree):
            # Use X_train_tree for Prophet, X_train_sarimax for SARIMAX
            X_train_to_use = X_train_sarimax if model_name == "SARIMAX" else X_train_tree

            X_train_fold, X_val_fold = X_train_to_use.iloc[train_idx], X_train_to_use.iloc[val_idx]
            y_train_fold, y_val_fold = y_train_horizon.iloc[train_idx], y_train_horizon.iloc[val_idx]

            if model_name == "SARIMAX":
                # statsmodels needs to be re-initialized for each fold with the correct data
                model_instance = sm.tsa.SARIMAX(
                    endog=y_train_fold, exog=X_train_fold, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7)
                ).fit(disp=False)
                y_pred_fold = model_instance.predict(
                    start=X_val_fold.index[0], end=X_val_fold.index[-1], exog=X_val_fold
                )
            else:  # For ProphetWrapper and other scikit-learn compatible models
                model_obj.fit(X_train_fold, y_train_fold)
                y_pred_fold = model_obj.predict(X_val_fold)

            fold_scores["r2"].append(r2_score(y_val_fold, y_pred_fold))
            fold_scores["rmse"].append(np.sqrt(mean_squared_error(y_val_fold, y_pred_fold)))
            fold_scores["mae"].append(mean_absolute_error(y_val_fold, y_pred_fold))
            fold_scores["mape"].append(_calculate_mape(y_val_fold.values, y_pred_fold))

        model_results[f"t+{h}"] = {"scores": fold_scores}

    advanced_ts_results[model_name] = model_results

joblib.dump(advanced_ts_results, "experiments/bakeoff_results_advanced_ts.pkl")
print("Advanced time-series model results saved to 'experiments/bakeoff_results_advanced_ts.pkl'")


In [ ]:
# sandbox_notebook.ipynb - Deep Learning (LSTM) Bake-Off Script

# --- 1. Imports for Deep Learning ---
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
import gc  # Garbage Collector

# (Assumes other necessary imports like os, joblib, pandas, numpy are present)


# --- 2. Define a scikit-learn compatible Wrapper for the Keras LSTM Model ---
class LSTMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_timesteps=7, n_features=0, epochs=50, batch_size=32, validation_split=0.1, verbose=0):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.verbose = verbose
        self.model = None
        self.scaler = RobustScaler()  # Each model needs its own scaler fit on its training data

    def _create_sequences(self, X, y):
        """Helper to transform tabular data into 3D sequences for LSTM."""
        X_seq, y_seq = [], []
        for i in range(len(X) - self.n_timesteps):
            X_seq.append(X[i : (i + self.n_timesteps)])
            y_seq.append(y[i + self.n_timesteps - 1])  # Target corresponds to the last day in the sequence
        return np.array(X_seq), np.array(y_seq)

    def fit(self, X, y):
        self.n_features = X.shape[1]

        # 1. Scale the data
        X_scaled = self.scaler.fit_transform(X)

        # 2. Reshape into 3D sequences
        X_seq, y_seq = self._create_sequences(X_scaled, y.values)

        if len(X_seq) == 0:
            print("Warning: Not enough data to create sequences with the given timesteps. Skipping fit.")
            return self

        # 3. Build the LSTM model architecture
        tf.keras.backend.clear_session()
        self.model = Sequential([
            LSTM(64, activation="relu", input_shape=(self.n_timesteps, self.n_features), return_sequences=True),
            Dropout(0.2),
            LSTM(32, activation="relu"),
            Dropout(0.2),
            Dense(16, activation="relu"),
            Dense(1),  # Output layer
        ])
        self.model.compile(optimizer="adam", loss="mse")

        # 4. Train the model
        early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
        self.model.fit(
            X_seq,
            y_seq,
            epochs=self.epochs,
            batch_size=self.batch_size,
            validation_split=self.validation_split,
            callbacks=[early_stopping],
            verbose=self.verbose,
        )
        return self

    def predict(self, X):
        if self.model is None:
            return np.zeros(len(X))

        # Scale and reshape the prediction data
        X_scaled = self.scaler.transform(X)

        # We need to create sequences, but only predict for the valid range
        X_seq, _ = self._create_sequences(X_scaled, np.zeros(len(X_scaled)))  # Dummy y

        if len(X_seq) == 0:
            # Handle cases where X is too short to form a full sequence
            return np.full(len(X), np.nan)

        predictions = self.model.predict(X_seq, verbose=self.verbose).flatten()

        # The predictions align with the END of each sequence. We need to pad the start.
        padding = np.full(self.n_timesteps, np.nan)
        return np.concatenate([padding, predictions])


# --- 3. Run the Bake-Off for the LSTM ---
print("\n--- Running Bake-Off for LSTM Model ---")
# Use a smaller subset of features for LSTM to keep training time manageable
# and focus on the most impactful variables.
lstm_features = [
    "temp_lag_1",
    "temp_lag_2",
    "temp_lag_3",
    "temp_lag_7",
    "sin_day_of_year",
    "cos_day_of_year",
    "solarradiation",
    "cloudcover",
    "humidity",
    "precip",
    "windspeed",
    "dew",
]
X_train_lstm = X_train_tree[lstm_features]

dl_models = {
    # Using a 7-day lookback window (timesteps) for the LSTM.
    "LSTM": LSTMWrapper(n_timesteps=7)
}
dl_results = {}

# We need another custom loop because the data needs to be reshaped
# and predictions need to be handled carefully.
for model_name, model_obj in dl_models.items():
    model_results = {}
    for h in tqdm(config.HORIZONS, desc=f"Training {model_name}"):
        target_col = f"target_temp_t+{h}"
        y_train_horizon = y_train[target_col]
        fold_scores = {"r2": [], "rmse": [], "mae": [], "mape": []}

        for train_idx, val_idx in tscv_splitter.split(X_train_lstm):
            X_train_fold, X_val_fold = X_train_lstm.iloc[train_idx], X_train_lstm.iloc[val_idx]
            y_train_fold, y_val_fold = y_train_horizon.iloc[train_idx], y_train_horizon.iloc[val_idx]

            model_instance = LSTMWrapper(n_timesteps=7)
            model_instance.fit(X_train_fold, y_train_fold)
            y_pred_fold_raw = model_instance.predict(X_val_fold)

            # Important: The wrapper returns NaNs at the start. We must align predictions with true values.
            valid_preds_mask = ~np.isnan(y_pred_fold_raw)
            y_pred_fold_aligned = y_pred_fold_raw[valid_preds_mask]
            y_val_fold_aligned = y_val_fold.values[valid_preds_mask]

            if len(y_pred_fold_aligned) > 0:
                fold_scores["r2"].append(r2_score(y_val_fold_aligned, y_pred_fold_aligned))
                fold_scores["rmse"].append(np.sqrt(mean_squared_error(y_val_fold_aligned, y_pred_fold_aligned)))
                fold_scores["mae"].append(mean_absolute_error(y_val_fold_aligned, y_pred_fold_aligned))
                fold_scores["mape"].append(_calculate_mape(y_val_fold_aligned, y_pred_fold_aligned))

            # Clean up memory
            del model_instance
            gc.collect()

        model_results[f"t+{h}"] = {"scores": fold_scores}
    dl_results[model_name] = model_results

joblib.dump(dl_results, "experiments/bakeoff_results_dl.pkl")
print("Deep Learning model results saved to 'experiments/bakeoff_results_dl.pkl'")


In [ ]:
# sandbox_notebook.ipynb - Ensemble Bake-Off Script

# --- 1. Imports ---
from sklearn.linear_model import RidgeCV
from catboost import CatBoostRegressor

# --- 2. Define Models for Ensemble Bake-Off ---
# The SimpleAveragingEnsemble is handled by a special case in the runner function,
# so we pass its components in a dictionary.
ensemble_models = {
    "SimpleAveragingEnsemble": {
        "linear": RidgeCV(cv=TimeSeriesSplit(n_splits=5)),
        "tree": CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
    },
    "RidgeCV_CatBoost_with_RidgeCV": train.StackingEnsemble(
        base_model_linear=RidgeCV(cv=TimeSeriesSplit(n_splits=5)),
        base_model_tree=CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
    ),
    "RidgeCV_CatBoost_with_ElasticNeCV": train.StackingEnsemble(
        base_model_linear=RidgeCV(cv=TimeSeriesSplit(n_splits=5)),
        base_model_tree=CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
        meta_learner=ElasticNetCV(cv=TimeSeriesSplit(n_splits=3)),
    ),
    "ElasticNetCV_CatBoost_with_RidgeCV": train.StackingEnsemble(
        base_model_linear=ElasticNetCV(cv=TimeSeriesSplit(n_splits=5)),
        base_model_tree=CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
    ),
    "ElasticNet_CatBoost_with_ElasticNetCV": train.StackingEnsemble(
        base_model_linear=ElasticNetCV(cv=TimeSeriesSplit(n_splits=5)),
        base_model_tree=CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
        meta_learner=ElasticNetCV(cv=TimeSeriesSplit(n_splits=3)),
    ),
    "ElasticNetCV_LGBM_with_RidgeCV": train.StackingEnsemble(
        base_model_linear=ElasticNetCV(cv=TimeSeriesSplit(n_splits=5)),
        base_model_tree=LGBMRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=-1),
    ),
}

# --- 3. Run Ensemble Bake-Off ---
print("\n--- Running Bake-Off for Ensemble Models ---")
# Assuming tscv_splitter is already defined from the previous bake-off cell
ensemble_results = train.run_ensemble_bakeoff(
    ensembles_to_run=ensemble_models,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(ensemble_results, "experiments/bakeoff_results_ensemble.pkl")
print("Ensemble model results saved to 'experiments/bakeoff_results_ensemble.pkl'")


### **Diagnostic scripts - Train full then test, no mean CV evaluation**

In [ ]:
# sandbox_notebook.ipynb - Diagnostic Script for Full Train/Test Evaluation

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


# --- Helper Functions (copied from utils/evaluation.py for self-containment) ---
def _calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-8))) * 100


def calculate_all_metrics(y_true, y_pred):
    return {
        "R²": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE": _calculate_mape(y_true, y_pred),
    }


# --- 1. Define Finalist Models ---
# Using the same definitions as the bake-off for consistency
finalist_linear_models = {
    "LinearRegression": LinearRegression(n_jobs=-1),
    "RidgeCV": RidgeCV(),  # CV is not used here, but we keep the object for consistency
    "ElasticNetCV": ElasticNetCV(random_state=config.GLOBAL_RANDOM_SEED, n_jobs=-1),
}

finalist_tree_models = {
    "CatBoostRegressor": CatBoostRegressor(random_state=config.GLOBAL_RANDOM_SEED, verbose=0),
    "LGBMRegressor": LGBMRegressor(random_state=config.GLOBAL_RANDOM_SEED, n_jobs=-1, verbosity=-1),
}

all_finalists = {**finalist_linear_models, **finalist_tree_models}

# --- 2. Run Full Evaluation Loop ---
print("--- RUNNING DIAGNOSTIC: EVALUATION ON FULL TRAIN/TEST SPLIT ---\n")

all_results_summary = []

for model_name, model_obj in all_finalists.items():
    print(f"--- Evaluating Model: {model_name} ---")

    # Select the correct feature set
    is_linear = model_name in finalist_linear_models
    X_train_to_use = X_train_linear if is_linear else X_train_tree
    X_test_to_use = X_test_linear if is_linear else X_test_tree

    horizon_metrics = []

    for h in config.HORIZONS:
        target_col = f"target_temp_t+{h}"

        # 1. Fit model on the ENTIRE training set
        model_obj.fit(X_train_to_use, y_train[target_col])

        # 2. Predict ONCE on the test set
        y_pred_test = model_obj.predict(X_test_to_use)

        # 3. Calculate all metrics
        metrics = calculate_all_metrics(y_test[target_col], y_pred_test)
        metrics["Horizon"] = f"t+{h}"
        horizon_metrics.append(metrics)

    # --- 3. Display Results for the current model ---
    df_model_results = pd.DataFrame(horizon_metrics)

    # Calculate average and add it as a new row
    avg_metrics = df_model_results.mean(numeric_only=True)
    avg_metrics["Horizon"] = "Average"
    # pd.concat is deprecated, use _append
    df_model_results = df_model_results._append(avg_metrics, ignore_index=True)

    df_model_results = df_model_results.set_index("Horizon")

    print(df_model_results.to_string(float_format="%.4f"))
    print("\n" + "=" * 60 + "\n")

    # Store for final summary if needed
    all_results_summary.append({"model": model_name, "results": df_model_results})


In [ ]:
# sandbox_notebook.ipynb - Diagnostic Script for Tuned Trees & Ensembles
from sklearn import clone


# --- 1. Define Sensible Heuristic Defaults for Tree Models ---
# These are more generous than the library defaults to give them a better chance.
# Based on common practices for similar datasets.
sensible_lgbm_params = {
    "n_estimators": 500,
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": 7,
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "n_jobs": -1,
    "random_state": config.GLOBAL_RANDOM_SEED,
    "verbosity": -1,
}

sensible_catboost_params = {
    "iterations": 1000,
    "learning_rate": 0.05,
    "depth": 7,
    "l2_leaf_reg": 3,
    "verbose": 0,
    "random_state": config.GLOBAL_RANDOM_SEED,
}

tuned_tree_models = {
    "LGBMRegressor_tuned": LGBMRegressor(**sensible_lgbm_params),
    "CatBoostRegressor_tuned": CatBoostRegressor(**sensible_catboost_params),
}

# --- 2. Define Base Models for Ensembling ---
# We will use the best linear and the new tuned tree model.
base_models_for_ensemble = {
    "RidgeCV": finalist_linear_models["RidgeCV"],
    "LGBMRegressor_tuned": tuned_tree_models["LGBMRegressor_tuned"],
}

# --- 3. Create Ensemble Models to Test ---
from sklearn.ensemble import StackingRegressor

ensemble_models = {}


# Simple Averaging Ensemble
class AveragingEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = models
        self.fitted_models_ = []

    def fit(self, X, y):
        self.fitted_models_ = [clone(model).fit(X, y) for name, model in self.models.items()]
        return self

    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.fitted_models_])
        return np.mean(predictions, axis=1)


ensemble_models["AvgEnsemble(Ridge+LGBM)"] = AveragingEnsemble(models=base_models_for_ensemble)

# Stacking Ensemble
# The ChampionForecastStack is designed for this, but for a quick diagnostic,
# sklearn's StackingRegressor is sufficient and clear.
stacking_ensemble = StackingRegressor(
    estimators=list(base_models_for_ensemble.items()), final_estimator=RidgeCV(), n_jobs=-1
)
ensemble_models["StackingEnsemble(Ridge+LGBM)"] = stacking_ensemble


# --- 4. Combine Models and Run Evaluation ---
models_to_evaluate = {**tuned_tree_models, **ensemble_models}

print("\n" + "=" * 80 + "\n")
print("--- RUNNING DIAGNOSTIC: EVALUATION FOR TUNED TREES & ENSEMBLES ---\n")

for model_name, model_obj in models_to_evaluate.items():
    print(f"--- Evaluating Model: {model_name} ---")

    # All these models use the full tree feature set
    X_train_to_use = X_train_tree
    X_test_to_use = X_test_tree

    horizon_metrics = []

    for h in config.HORIZONS:
        target_col = f"target_temp_t+{h}"

        # Fit model on the ENTIRE training set
        model_obj.fit(X_train_to_use, y_train[target_col])

        # Predict ONCE on the test set
        y_pred_test = model_obj.predict(X_test_to_use)

        # Calculate all metrics
        metrics = calculate_all_metrics(y_test[target_col], y_pred_test)
        metrics["Horizon"] = f"t+{h}"
        horizon_metrics.append(metrics)

    # Display Results for the current model
    df_model_results = pd.DataFrame(horizon_metrics).set_index("Horizon")
    avg_metrics = df_model_results.mean(numeric_only=True)
    avg_metrics.name = "Average"
    df_model_results = df_model_results._append(avg_metrics)

    print(df_model_results.to_string(float_format="%.4f"))
    print("\n" + "=" * 60 + "\n")


### **HPOs**

#### **CatBoost - OLD AVERAGE ONE, discarded**

In [ ]:
# # sandbox_notebook.ipynb - HPO Study 1: CatBoostRegressor 
# This was the old average (one set of params for all horizons) tuning

# # --- 1. Imports ---
# import optuna
# from sklearn.model_selection import TimeSeriesSplit

# # Assuming all utility functions and data are loaded
# import utils.training as train

# # --- 2. Setup Study ---
# STUDY_NAME_CATBOOST = "hpo_catboost_regressor"
# N_TRIALS_CATBOOST = 175  # Adjust as needed based on time constraints

# # --- 3. Define the Objective Function ---
# # We create a dictionary for the feature sets, even though this model only uses one.
# # This keeps the function signature consistent for our objective factory.
# X_train_for_tuning = {
#     "linear": X_train_linear,  # Placeholder, not used by this model
#     "tree": X_train_tree,
# }

# catboost_objective = train.create_optuna_objective(
#     model_name="CatBoostRegressor",
#     X_train_dict=X_train_for_tuning,
#     y_train=y_train,
#     cv_splitter=TimeSeriesSplit(n_splits=5),
#     horizons=config.HORIZONS,
# )

# # --- 4. Run the Study ---
# print(f"--- Running Optuna Study for CatBoostRegressor ({N_TRIALS_CATBOOST} trials) ---")
# # To monitor in real-time, run this in your terminal:
# # optuna-dashboard sqlite:///experiments/hpo_catboost_regressor.db
# catboost_study = train.run_optuna_study(
#     study_name=STUDY_NAME_CATBOOST, objective_func=catboost_objective, n_trials=N_TRIALS_CATBOOST
# )

# # --- 5. Save the Best Parameters ---
# train.save_tuned_params(
#     study=catboost_study, horizons=config.HORIZONS, file_path="experiments/tuned_params_CatBoostRegressor.pkl"
# )

# print("\n--- CatBoostRegressor Tuning Complete ---")


#### **LGBM - Per-horizon**

In [ ]:
# sandbox_notebook.ipynb - HPO Study 2: LGBM

# --- HPO CONFIGURATION ---
TUNE_PER_HORIZON = True
N_TRIALS = 250  # Total trials if False, trials per horizon if True

# --- 2. Data Setup ---
X_train_for_tuning = {"linear": X_train_linear, "tree": X_train_tree}
tscv_splitter = TimeSeriesSplit(n_splits=5)

# --- 3. Run Study for LGBMRegressor ---
STUDY_NAME_LGBM = "hpo_lgbm_regressor"
lgbm_custom_space = {
    # --- Core Learning Parameters (Narrowed) ---
    "learning_rate": {"type": "float", "low": 0.005, "high": 0.04, "log": True},
    "n_estimators": {"type": "int", "low": 300, "high": 2000},
    # --- Tree Structure Parameters (Focused) ---
    "num_leaves": {"type": "int", "low": 20, "high": 60},  # Kept the same, good range
    "max_depth": {"type": "int", "low": 3, "high": 6},  # Narrowed significantly
    # --- Regularization Parameters (Constrained) ---
    "lambda_l1": {"type": "float", "low": 1e-8, "high": 1.0, "log": True},  # Reduced upper bound
    "lambda_l2": {"type": "float", "low": 1e-8, "high": 1.0, "log": True},  # Reduced upper bound
    # --- Bagging/Subsampling Parameters (Kept for robustness) ---
    "feature_fraction": {"type": "float", "low": 0.5, "high": 0.9},
    "bagging_fraction": {"type": "float", "low": 0.5, "high": 0.9},
    "bagging_freq": {"type": "int", "low": 1, "high": 7},
    # --- Other Structural Parameters ---
    "min_child_samples": {"type": "int", "low": 5, "high": 50},
}
if TUNE_PER_HORIZON:
    per_horizon_studies = {}
    for h in config.HORIZONS:
        print(f"--- Tuning LGBMRegressor for HORIZON t+{h} ({N_TRIALS} trials) ---")
        objective = train.create_optuna_objective(
            model_name="LGBMRegressor",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            custom_search_space=lgbm_custom_space,
        )
        study = train.run_optuna_study(f"{STUDY_NAME_LGBM}_h{h}", objective, n_trials=N_TRIALS)
        per_horizon_studies[h] = study
    train.save_tuned_params(None, "experiments/tuned_params_LGBMRegressor.pkl", per_horizon_studies)
else:
    print(f"--- Tuning LGBMRegressor for AVERAGE PERFORMANCE ({N_TRIALS} trials) ---")
    objective = train.create_optuna_objective(
        model_name="LGBMRegressor",
        X_train_dict=X_train_for_tuning,
        y_train=y_train,
        cv_splitter=tscv_splitter,
        horizon_to_tune=None,
        custom_search_space=lgbm_custom_space,
    )
    study = train.run_optuna_study(STUDY_NAME_LGBM, objective, n_trials=N_TRIALS)
    train.save_tuned_params(study, "experiments/tuned_params_LGBMRegressor.pkl")


#### **CatBoost - Per-horizon**

In [ ]:
def create_catboost_targeted_spaces(base_params: dict) -> list:
    """
    Generates a list of five targeted hyperparameter search spaces for CatBoost,
    one for each forecast horizon, based on a set of well-performing base parameters.

    The strategy is to start with a complex search space for t+1 and progressively
    simplify it for longer horizons to combat overfitting on weaker signals.
    """
    targeted_spaces = []

    # Base values from our best "average" trial
    base_lr = base_params["learning_rate"]
    base_iter = base_params["iterations"]
    base_depth = base_params["depth"]

    for h in range(1, 6):  # For horizons t+1 through t+5
        # Principle: Complexity and precision decrease as horizon increases

        # Learning Rate: Allow slightly lower for t+1, slightly higher for t+5
        lr_low = max(0.005, base_lr - 0.007)
        lr_high = min(0.05, base_lr + 0.01 * h)

        # Iterations: High for t+1, decreasing for longer horizons
        iter_low = max(200, base_iter - 100 * h)
        iter_high = max(400, base_iter + 400 - 150 * (h - 1))

        # Depth: Higher for t+1, strictly decreasing for longer horizons
        depth_low = max(3, base_depth - (h // 2))
        depth_high = min(8, base_depth + 2 - (h // 2))

        space = {
            "learning_rate": {"type": "float", "low": lr_low, "high": lr_high, "log": True},
            "iterations": {"type": "int", "low": int(iter_low), "high": int(iter_high)},
            "depth": {"type": "int", "low": depth_low, "high": depth_high},
            # Keep other parameters with a reasonably wide search space as they are less critical
            "l2_leaf_reg": {"type": "float", "low": 1e-5, "high": 10.0, "log": True},
            "random_strength": {"type": "float", "low": 1e-5, "high": 10.0, "log": True},
            "bagging_temperature": {"type": "float", "low": 0.0, "high": 1.0},
        }
        targeted_spaces.append(space)

    return targeted_spaces


best_avg_catboost_params = {
    "learning_rate": 0.0149,
    "iterations": 528,
    "depth": 5,
}

catboost_targeted_search_spaces = create_catboost_targeted_spaces(best_avg_catboost_params)


In [ ]:
# sandbox_notebook.ipynb - HPO Study 3: CatBoost per-horizon

# --- HPO CONFIGURATION ---
TUNE_PER_HORIZON = True
N_TRIALS = 200  # Total trials if False, trials per horizon if True

# --- 2. Data Setup ---
X_train_for_tuning = {"linear": X_train_linear, "tree": X_train_tree}
tscv_splitter = TimeSeriesSplit(n_splits=5)

# --- 3. Run Study for CatBoostRegressor ---
STUDY_NAME_CATBOOST = "hpo_catboost_regressor"
if TUNE_PER_HORIZON:
    per_horizon_studies = {}
    for h in config.HORIZONS:
        print(f"--- Tuning CatBoostRegressor for HORIZON t+{h} ({N_TRIALS} trials) ---")
        objective = train.create_optuna_objective(
            model_name="CatBoostRegressor",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            custom_search_space=catboost_targeted_search_spaces[h - 1],
        )
        study = train.run_optuna_study(f"{STUDY_NAME_CATBOOST}_h{h}", objective, n_trials=N_TRIALS)
        per_horizon_studies[h] = study
    train.save_tuned_params(None, "experiments/tuned_params_CatBoostRegressor.pkl", per_horizon_studies)
else:
    print(f"--- Tuning CatBoostRegressor for AVERAGE PERFORMANCE ({N_TRIALS} trials) ---")
    objective = train.create_optuna_objective(
        model_name="CatBoostRegressor",
        X_train_dict=X_train_for_tuning,
        y_train=y_train,
        cv_splitter=tscv_splitter,
        horizon_to_tune=None,
        custom_search_space=catboost_targeted_search_spaces[h - 1],
    )
    study = train.run_optuna_study(STUDY_NAME_CATBOOST, objective, n_trials=N_TRIALS)
    train.save_tuned_params(study, "experiments/tuned_params_CatBoostRegressor.pkl")


#### **Preliminary evaluation of tuned base models**

In [ ]:
tuned_params_paths = {
    "CatBoostRegressor": "experiments/tuned_params_CatBoostRegressor_v2.pkl",
    "LGBMRegressor": "./experiments/tuned_params_LGBMRegressor_v3.pkl",
}

# Configuration for all models we want to evaluate in this run
models_to_evaluate = {
    # Standalone Tuned Tree Models
    "Tuned_CatBoost_v2_weighted_CV": {
        "type": "single",
        "tree_model_name": "CatBoostRegressor",
    },
    "Tuned_LGBM_v2_weighted_CV": {
        "type": "single",
        "tree_model_name": "LGBMRegressor",
    },
    # # --- Tuned Simple Averaging Ensembles ---
    # "Avg_Ridge_TunedCatBoost": {
    #     "type": "simple_averaging",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_name": "CatBoostRegressor",
    # },
    # "Avg_ElasticNet_TunedLGBM": {
    #     "type": "simple_averaging",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_name": "LGBMRegressor",
    # },
    # # --- Tuned Stacking Ensembles ---
    # # CatBoost combinations
    # "Stack_Ridge_TunedCatBoost_RidgeMeta": {
    #     "type": "stacking",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_name": "CatBoostRegressor",
    #     "meta_learner_class": RidgeCV,
    # },
    # "Stack_ElasticNet_TunedCatBoost_RidgeMeta": {
    #     "type": "stacking",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_name": "CatBoostRegressor",
    #     "meta_learner_class": RidgeCV,
    # },
    # "Stack_ElasticNet_TunedCatBoost_ElasticNetMeta": {
    #     "type": "stacking",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_name": "CatBoostRegressor",
    #     "meta_learner_class": ElasticNetCV,
    # },
    # # LGBM combinations
    # "Stack_Ridge_TunedLGBM_RidgeMeta": {
    #     "type": "stacking",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_name": "LGBMRegressor",
    #     "meta_learner_class": RidgeCV,
    # },
    # "Stack_ElasticNet_TunedLGBM_RidgeMeta": {
    #     "type": "stacking",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_name": "LGBMRegressor",
    #     "meta_learner_class": RidgeCV,
    # },
    # "Stack_ElasticNet_TunedLGBM_ElasticNetMeta": {
    #     "type": "stacking",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_name": "LGBMRegressor",
    #     "meta_learner_class": ElasticNetCV,
    # },
}

# --- 3. Run the Evaluation ---
print("\n--- Evaluating Performance of Tuned Models and Ensembles ---")
tuned_evaluation_results = eval.evaluate_tuned_models(
    models_to_evaluate=models_to_evaluate,
    tuned_params_paths=tuned_params_paths,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
filename_of_tuned_models = "experiments/evaluation_results_tuned_single_models_v2_weighted_CV.pkl"
joblib.dump(tuned_evaluation_results, filename_of_tuned_models)
print(f"Tuned model evaluation results saved to '{filename_of_tuned_models}'")


In [ ]:
results_files_to_load = [
    "experiments/bakeoff_results_linear.pkl",
    "experiments/bakeoff_results_tree.pkl",
    # "experiments/bakeoff_results_ensemble.pkl",
    # "experiments/evaluation_results_tuned_preliminary_ensemble_models.pkl",
    "experiments/evaluation_results_tuned_single_models_v1_mean.pkl",
    "experiments/evaluation_results_tuned_single_models_v2_weighted_CV.pkl",
]

df_leaderboard_final = eval.generate_leaderboard_data(results_files=results_files_to_load)
eval.display_leaderboard_stage1(df_leaderboard_final)


#### **LGBM - Per-horizon re-tune with weighted CV**

In [ ]:
# sandbox_notebook.ipynb - Cell 1: Setup & Configuration

# --- 2. HPO CONFIGURATION ---
NUM_RUNS_PER_HORIZON = 7
N_TRIALS_PER_RUN = 100
CV_WEIGHTS = np.array([0.10, 0.15, 0.20, 0.25, 0.30])

# --- 3. Data & Splitter Setup ---
X_train_for_tuning = {"linear": X_train_linear, "tree": X_train_tree}
tscv_splitter = TimeSeriesSplit(n_splits=5)

# --- 4. Define Final Tuned LGBM Parameters (from single-model run) ---
# This is our reference point for creating targeted search spaces.
lgbm_single_model_best_params = {
    1: {
        "learning_rate": 0.0053,
        "n_estimators": 852,
        "num_leaves": 54,
        "max_depth": 5,
        "lambda_l1": 1.4e-05,
        "lambda_l2": 2.0e-05,
        "feature_fraction": 0.75,
        "bagging_fraction": 0.50,
        "bagging_freq": 4,
        "min_child_samples": 24,
    },
    2: {
        "learning_rate": 0.0053,
        "n_estimators": 722,
        "num_leaves": 20,
        "max_depth": 4,
        "lambda_l1": 0.122,
        "lambda_l2": 8.4e-05,
        "feature_fraction": 0.73,
        "bagging_fraction": 0.51,
        "bagging_freq": 3,
        "min_child_samples": 33,
    },
    3: {
        "learning_rate": 0.0060,
        "n_estimators": 589,
        "num_leaves": 33,
        "max_depth": 3,
        "lambda_l1": 1.7e-08,
        "lambda_l2": 2.7e-05,
        "feature_fraction": 0.69,
        "bagging_fraction": 0.51,
        "bagging_freq": 1,
        "min_child_samples": 42,
    },
    4: {
        "learning_rate": 0.0100,
        "n_estimators": 365,
        "num_leaves": 26,
        "max_depth": 3,
        "lambda_l1": 7.1e-08,
        "lambda_l2": 0.00015,
        "feature_fraction": 0.56,
        "bagging_fraction": 0.53,
        "bagging_freq": 7,
        "min_child_samples": 41,
    },
    5: {
        "learning_rate": 0.0086,
        "n_estimators": 490,
        "num_leaves": 25,
        "max_depth": 3,
        "lambda_l1": 0.035,
        "lambda_l2": 0.0033,
        "feature_fraction": 0.50,
        "bagging_fraction": 0.53,
        "bagging_freq": 6,
        "min_child_samples": 29,
    },
}


# --- 5. Function to Generate Targeted Spaces ---
def create_targeted_search_space(base_params: Dict, buffer: float = 0.3) -> Dict:
    """Creates a targeted Optuna search space around a set of base parameters."""
    space = {}
    for param, value in base_params.items():
        if isinstance(value, int) and param not in ["bagging_freq", "max_depth"]:
            low = max(1, int(value * (1 - buffer)))
            high = int(value * (1 + buffer))
            space[param] = {"type": "int", "low": low, "high": high}
        elif isinstance(value, float):
            low = max(1e-9, value * (1 - buffer))
            high = (
                min(1.0, value * (1 + buffer))
                if param in ["feature_fraction", "bagging_fraction"]
                else value * (1 + buffer)
            )
            space[param] = {
                "type": "float",
                "low": low,
                "high": high,
                "log": param in ["learning_rate", "lambda_l1", "lambda_l2"],
            }
        # For categorical or fixed-range params, define them explicitly
        elif param == "max_depth":
            space[param] = {"type": "int", "low": max(2, value - 1), "high": value + 2}
        elif param == "bagging_freq":
            space[param] = {"type": "int", "low": max(1, value - 2), "high": value + 2}
    return space


# --- 6. Generate the Dictionary of Targeted Spaces ---
lgbm_targeted_spaces = {h: create_targeted_search_space(lgbm_single_model_best_params[h]) for h in config.HORIZONS}


In [ ]:
# sandbox_notebook.ipynb - Cell 2: Re-tune Single LGBM

print("\n" + "=" * 50)
print("  STARTING: Re-tuning Single LGBMRegressor with Weighted CV")
print("=" * 50 + "\n")

STUDY_NAME_LGBM_WEIGHTED = "hpo_lgbm_regressor_weighted_v3"
lgbm_weighted_studies = {}
for h in config.HORIZONS:
    for i in range(NUM_RUNS_PER_HORIZON):
        run_name = f"{STUDY_NAME_LGBM_WEIGHTED}_h{h}_run{i + 1}"
        print(f"--- Running Study: {run_name} ({N_TRIALS_PER_RUN} trials) ---")
        objective = train.create_optuna_objective(
            model_name="LGBMRegressor",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            cv_weights=CV_WEIGHTS,
            custom_search_space=lgbm_targeted_spaces[h],
        )
        train.run_optuna_study(run_name, objective, n_trials=N_TRIALS_PER_RUN)

# Consolidate results for the re-tuned LGBM
train.consolidate_best_params(
    base_study_name=STUDY_NAME_LGBM_WEIGHTED,
    horizons=config.HORIZONS,
    num_runs=NUM_RUNS_PER_HORIZON,
    output_file_path="experiments/tuned_params_LGBMRegressor_v3.pkl",
)


#### **LGBM - Per-horizon re-tune with weighted CV AND `linear_tree=True`**

In [ ]:
# sandbox_notebook.ipynb - HPO Study: LGBM with linear_tree=True

# --- 1. Imports & Configuration ---
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import utils.training as train

# --- HPO CONFIGURATION ---
NUM_RUNS_PER_HORIZON = 2
N_TRIALS_PER_RUN = 190
CV_WEIGHTS = np.array([0.10, 0.15, 0.20, 0.25, 0.30])
X_train_for_tuning = {"linear": X_train_linear, "tree": X_train_tree}
tscv_splitter = TimeSeriesSplit(n_splits=5)

# --- 2. Define FINAL Targeted Search Spaces for linear_tree ---
# Based on deep analysis of the initial run and fANOVA plots.

lgbm_linear_tree_targeted_spaces = {
    # --- Horizon 1: Focus on n_estimators and bagging ---
    1: {
        "learning_rate": {"type": "float", "low": 0.0045, "high": 0.006, "log": True},
        "n_estimators": {"type": "int", "low": 900, "high": 1450},
        "num_leaves": {"type": "int", "low": 60, "high": 170},
        "max_depth": {"type": "int", "low": 3, "high": 5},  # Constrained
        "lambda_l1": {"type": "float", "low": 1e-8, "high": 1e-4, "log": True},  # Unimportant, constrain
        "lambda_l2": {"type": "float", "low": 5e-7, "high": 5e-5, "log": True},  # Unimportant, constrain
        "feature_fraction": {"type": "float", "low": 0.92, "high": 1},
        "bagging_fraction": {"type": "float", "low": 0.61, "high": 0.7},  # Focused range
        "bagging_freq": {"type": "int", "low": 1, "high": 3},
        "min_child_samples": {"type": "int", "low": 70, "high": 120},
    },
    # --- Horizon 2: Similar to H1, but slightly simpler ---
    2: {
        "learning_rate": {"type": "float", "low": 0.005, "high": 0.01, "log": True},
        "n_estimators": {"type": "int", "low": 500, "high": 1100},
        "num_leaves": {"type": "int", "low": 40, "high": 140},
        "max_depth": {"type": "int", "low": 3, "high": 5},
        "lambda_l1": {"type": "float", "low": 1e-8, "high": 1e-4, "log": True},
        "lambda_l2": {"type": "float", "low": 1e-8, "high": 0.1, "log": True},
        "feature_fraction": {"type": "float", "low": 0.6, "high": 0.8},
        "bagging_fraction": {"type": "float", "low": 0.42, "high": 0.62},
        "bagging_freq": {"type": "int", "low": 3, "high": 7},
        "min_child_samples": {"type": "int", "low": 60, "high": 100},
    },
    # --- Horizon 3: Focus on bagging_fraction and max_depth ---
    3: {
        "learning_rate": {"type": "float", "low": 0.006, "high": 0.012, "log": True},
        "n_estimators": {"type": "int", "low": 400, "high": 800},
        "num_leaves": {"type": "int", "low": 20, "high": 50},
        "max_depth": {"type": "int", "low": 3, "high": 4},
        "lambda_l1": {"type": "float", "low": 1e-6, "high": 0.1, "log": True},
        "lambda_l2": {"type": "float", "low": 1.0, "high": 4.0, "log": True},  # Focus on higher value
        "feature_fraction": {"type": "float", "low": 0.35, "high": 0.64},
        "bagging_fraction": {"type": "float", "low": 0.35, "high": 0.55},  # Focus on this key param
        "bagging_freq": {"type": "int", "low": 2, "high": 5},
        "min_child_samples": {"type": "int", "low": 80, "high": 120},
    },
    # --- Horizon 4: Return focus to n_estimators ---
    4: {
        "learning_rate": {"type": "float", "low": 0.005, "high": 0.01, "log": True},
        "n_estimators": {"type": "int", "low": 500, "high": 900},
        "num_leaves": {"type": "int", "low": 80, "high": 150},
        "max_depth": {"type": "int", "low": 3, "high": 6},
        "lambda_l1": {"type": "float", "low": 1e-8, "high": 0.1, "log": True},
        "lambda_l2": {"type": "float", "low": 1e-8, "high": 1e-4, "log": True},
        "feature_fraction": {"type": "float", "low": 0.4, "high": 0.6},
        "bagging_fraction": {"type": "float", "low": 0.3, "high": 0.5},
        "bagging_freq": {"type": "int", "low": 1, "high": 5},
        "min_child_samples": {"type": "int", "low": 70, "high": 110},
    },
    # --- Horizon 5: Focus on max_depth and bagging ---
    5: {
        "learning_rate": {"type": "float", "low": 0.007, "high": 0.015, "log": True},
        "n_estimators": {"type": "int", "low": 400, "high": 700},
        "num_leaves": {"type": "int", "low": 40, "high": 75},
        "max_depth": {"type": "int", "low": 3, "high": 4},  # Focus on this key param
        "lambda_l1": {"type": "float", "low": 1e-5, "high": 0.01, "log": True},
        "lambda_l2": {"type": "float", "low": 0.1, "high": 1.0, "log": True},
        "feature_fraction": {"type": "float", "low": 0.6, "high": 0.8},
        "bagging_fraction": {"type": "float", "low": 0.35, "high": 0.52},  # Focus on this key param
        "bagging_freq": {"type": "int", "low": 1, "high": 5},
        "min_child_samples": {"type": "int", "low": 70, "high": 110},
    },
}

# --- 3. Run the Tuning Study ---
STUDY_NAME_LGBM_LINEAR_TREE = "hpo_lgbm_regressor_weighted_CV_linear_tree_v2"
for h in [2, 3, 4, 5]:
    for i in range(NUM_RUNS_PER_HORIZON):
        run_name = f"{STUDY_NAME_LGBM_LINEAR_TREE}_h{h}_run{i + 1}"
        print(f"--- Running Study: {run_name} ({N_TRIALS_PER_RUN} trials) ---")

        objective = train.create_optuna_objective(
            model_name="LGBMRegressor",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            cv_weights=CV_WEIGHTS,
            custom_search_space=lgbm_linear_tree_targeted_spaces[h],
        )
        train.run_optuna_study(run_name, objective, n_trials=N_TRIALS_PER_RUN)

# --- 4. Consolidate Final Results ---
filename_to_save_tuned_params = "experiments/tuned_params_LGBMRegressor_v5.pkl"
train.consolidate_best_params(
    base_study_name=STUDY_NAME_LGBM_LINEAR_TREE,
    horizons=config.HORIZONS,
    num_runs=NUM_RUNS_PER_HORIZON,
    output_file_path=filename_to_save_tuned_params,
)


#### **Stacking ensemble 01 - Per-horizon with weighted CV**

In [ ]:
# sandbox_notebook.ipynb - Cell 3: Tune Stacking Ensemble (ElasticNet + LGBM + Ridge)

NUM_RUNS_PER_HORIZON = 4
N_TRIALS_PER_RUN = 50

print("\n" + "=" * 50)
print("  STARTING: Tuning Stack_ElasticNet_LGBM_RidgeMeta")
print("=" * 50 + "\n")

STUDY_NAME_STACK_LGBM = "hpo_stack_elasticnet_lgbm_ridge_v2"
stack_lgbm_studies = {}
for h in config.HORIZONS:
    for i in range(NUM_RUNS_PER_HORIZON):
        run_name = f"{STUDY_NAME_STACK_LGBM}_h{h}_run{i + 1}"
        print(f"--- Running Study: {run_name} ({N_TRIALS_PER_RUN} trials) ---")

        # We pass the full model name to the objective factory.
        # It will parse this to know it needs to build a StackingEnsemble
        # with an internal LGBMRegressor.
        objective = train.create_optuna_objective(
            model_name="Stacking_LGBM",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            cv_weights=CV_WEIGHTS,
            # CRUCIAL: We reuse the same targeted search space for this horizon.
            custom_search_space=lgbm_targeted_spaces[h],
        )
        train.run_optuna_study(run_name, objective, n_trials=N_TRIALS_PER_RUN)

# Consolidate results for this ensemble
train.consolidate_best_params(
    base_study_name=STUDY_NAME_STACK_LGBM,
    horizons=config.HORIZONS,
    num_runs=NUM_RUNS_PER_HORIZON,
    output_file_path="experiments/tuned_params_final_Stack_ElasticNet_LGBM_Ridge_v2.pkl",
)


#### **Stacking ensemble 02 - Per-horizon with weighted CV**

In [ ]:
# sandbox_notebook.ipynb - Cell 4: Define CatBoost Targeted Spaces

NUM_RUNS_PER_HORIZON = 2
N_TRIALS_PER_RUN = 20

# --- 1. Define Final Tuned CatBoost Parameters (from single-model run) ---
catboost_single_model_best_params = {
    1: {
        "learning_rate": 0.0175,
        "iterations": 605,
        "depth": 5,
        "l2_leaf_reg": 0.547,
        "random_strength": 0.033,
        "bagging_temperature": 0.356,
    },
    2: {
        "learning_rate": 0.0101,
        "iterations": 581,
        "depth": 6,
        "l2_leaf_reg": 0.227,
        "random_strength": 0.040,
        "bagging_temperature": 0.463,
    },
    3: {
        "learning_rate": 0.0094,
        "iterations": 594,
        "depth": 4,
        "l2_leaf_reg": 0.167,
        "random_strength": 4.037,
        "bagging_temperature": 0.154,
    },
    4: {
        "learning_rate": 0.0137,
        "iterations": 438,
        "depth": 4,
        "l2_leaf_reg": 0.184,
        "random_strength": 5.495,
        "bagging_temperature": 0.946,
    },
    5: {
        "learning_rate": 0.0203,
        "iterations": 349,
        "depth": 3,
        "l2_leaf_reg": 0.974,
        "random_strength": 9.870,
        "bagging_temperature": 0.764,
    },
}

# --- 2. Generate the Dictionary of Targeted Spaces for CatBoost ---
# We can reuse the same create_targeted_search_space function from the previous cell.
catboost_targeted_spaces = {
    h: create_targeted_search_space(catboost_single_model_best_params[h]) for h in config.HORIZONS
}

# sandbox_notebook.ipynb - Cell 5: Tune Stacking Ensemble (ElasticNet + CatBoost + Ridge)

print("\n" + "=" * 50)
print("  STARTING: Tuning Stack_ElasticNet_CatBoost_RidgeMeta")
print("=" * 50 + "\n")

STUDY_NAME_STACK_CATBOOST = "hpo_stack_elasticnet_catboost_ridge"
stack_catboost_studies = {}
for h in [2, 3, 4, 5]:
    for i in range(NUM_RUNS_PER_HORIZON):
        run_name = f"{STUDY_NAME_STACK_CATBOOST}_h{h}_run{i + 1}"
        print(f"--- Running Study: {run_name} ({N_TRIALS_PER_RUN} trials) ---")

        # Pass the full model name for the objective factory to parse
        objective = train.create_optuna_objective(
            model_name="Stacking_CatBoost",
            X_train_dict=X_train_for_tuning,
            y_train=y_train,
            cv_splitter=tscv_splitter,
            horizon_to_tune=h,
            cv_weights=CV_WEIGHTS,
            # Use the targeted search space for this horizon
            custom_search_space=catboost_targeted_spaces[h],
        )
        train.run_optuna_study(run_name, objective, n_trials=N_TRIALS_PER_RUN)

# Consolidate results for this ensemble
train.consolidate_best_params(
    base_study_name=STUDY_NAME_STACK_CATBOOST,
    horizons=config.HORIZONS,
    num_runs=NUM_RUNS_PER_HORIZON,
    output_file_path="experiments/tuned_params_final_Stack_ElasticNet_CatBoost_Ridge.pkl",
)


#### **MODELS EVALUATION**

##### **User guide**

| Parameter              | Data Type         | Description                                                                                             | Example                                             |
| :--------------------- | :---------------- | :------------------------------------------------------------------------------------------------------ | :-------------------------------------------------- |
| **`type`** (Required)  | `str`             | The category of model. Determines the logic used by `evaluate_models`.                                  | `"simple_averaging_tuned"`, `"stacking_tuned"`, `"single_tuned"`, `"oob"` |
| **`model_class`**      | `class`           | The scikit-learn compatible class for **OOB** (untuned) models.                                         | `LGBMRegressor`                                     |
| **`feature_set`**      | `str`             | **For OOB models only.** Specifies which feature set to use.                                            | `"linear"` or `"tree"`                              |
| **`linear_model_class`** | `class`         | **For ensembles.** The class for the linear base model.                                                 | `RidgeCV`, `ElasticNetCV`                           |
| **`tree_model_class`**   | `class`         | **For tuned models.** The class for the tree-based model (for parameter mapping).                       | `CatBoostRegressor`, `LGBMRegressor`                |
| **`meta_learner_class`** | `class`         | **For stacking ensembles.** The class for the meta-learner.                                             | `RidgeCV`                                           |
| **`params_path`**      | `str`             | **For tuned models.** The file path to the `.pkl` file containing the per-horizon tuned hyperparameters. | `"experiments/my_params.pkl"`                       |


**Examples:**

**1. To evaluate a new, untuned `XGBRegressor`:**
```python
"XGBRegressor_OOB": {
    "type": "oob",
    "model_class": XGBRegressor,
    "feature_set": "tree" # Use the full feature set
},
```

**2. To evaluate your `Stack_ElasticNet_LGBM_Ridge_Tuned` but using the `v1` parameters instead:**
```python
"Stack_ElasticNet_LGBM_Ridge_Tuned_v1": {
    "type": "stacking_tuned",
    "linear_model_class": ElasticNetCV,
    "tree_model_class": LGBMRegressor,
    "meta_learner_class": RidgeCV,
    "params_path": "experiments/tuned_params_final_Stack_ElasticNet_LGBM_Ridge.pkl" # Note the changed file path
},
```

**3. To evaluate a single, tuned `CatBoostRegressor` (using `v1` params):**
```python
"Tuned_CatBoost_v1": {
    "type": "single_tuned",
    "tree_model_class": CatBoostRegressor,
    "params_path": "experiments/tuned_params_CatBoostRegressor_v1.pkl"
},
```

In [ ]:
from sklearn.linear_model import ElasticNetCV, RidgeCV
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor


##### **Evaluate**

In [ ]:
models_to_evaluate = {
    # "Tuned_LGBM_v1_mean": {
    #     "type": "single_tuned",
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "./experiments/tuned_params_LGBMRegressor_v1.pkl",
    # },
    # "Tuned_LGBM_v2_weighted": {
    #     "type": "single_tuned",
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "./experiments/tuned_params_LGBMRegressor_v3.pkl",
    # },
    "Tuned_LGBM_v3_linear_tree": {
        "type": "single_tuned",
        "tree_model_class": LGBMRegressor,
        "params_path": "./experiments/tuned_params_LGBMRegressor_v5.pkl",
    },
    # "Avg_Ridge_LGBM_v1_mean": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "experiments/tuned_params_LGBMRegressor_v1.pkl",
    # },
    # "Avg_Ridge_LGBM_v2_weighted": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    # },
    # --- 1. Tuned Simple Averaging Ensembles ---
    # We will test all four logical combinations of our best linear and tuned tree models.
    # "Avg_Ridge_TunedCatBoost": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_class": CatBoostRegressor,
    #     "params_path": "experiments/tuned_params_CatBoostRegressor_v2.pkl",  # Using CatBoost v2
    # },
    # "Avg_ElasticNet_TunedCatBoost": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_class": CatBoostRegressor,
    #     "params_path": "experiments/tuned_params_CatBoostRegressor_v2.pkl",
    # },
    # "Avg_Ridge_TunedLGBM": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",  # Using LGBM v3
    # },
    # "Avg_ElasticNet_TunedLGBM": {
    #     "type": "simple_averaging_tuned",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_class": LGBMRegressor,
    #     "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    # },
    # # --- 2. Holistically Tuned Stacking Ensembles ---
    # # These models were tuned as a complete unit.
    # "Stack_ElasticNet_CatBoost_Ridge_Tuned": {
    #     "type": "stacking_tuned",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_class": CatBoostRegressor,
    #     "meta_learner_class": RidgeCV,
    #     "params_path": "experiments/tuned_params_final_Stack_ElasticNet_CatBoost_Ridge.pkl",
    # },
    # "Stack_ElasticNet_LGBM_Ridge_Tuned": {
    #     "type": "stacking_tuned",
    #     "linear_model_class": ElasticNetCV,
    #     "tree_model_class": LGBMRegressor,
    #     "meta_learner_class": RidgeCV,
    #     "params_path": "experiments/tuned_params_final_Stack_ElasticNet_LGBM_Ridge_v2.pkl",  # Using LGBM Stacker v2
    # },
}


In [ ]:
filename_of_evaluated_model_yes_long_name_i_know = "experiments/evaluation_results_tuned_LGBM_v3_linear_tree.pkl"

print("\n--- Running Final Evaluation of Tuned Ensembles ---")
final_tuned_ensemble_results = eval.evaluate_models(
    models_to_evaluate=models_to_evaluate,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(final_tuned_ensemble_results, filename_of_evaluated_model_yes_long_name_i_know)
print(f"Tuned ensemble evaluation results saved to '{filename_of_evaluated_model_yes_long_name_i_know}'")


##### **Display leaderboard**

In [ ]:
results_files_to_load = [
    "experiments/bakeoff_results_linear.pkl",
    # "experiments/bakeoff_results_tree.pkl",
    # "experiments/bakeoff_results_ensemble.pkl",
    # "experiments/evaluation_results_tuned_preliminary_ensemble_models.pkl",
    "experiments/evaluation_results_tuned_single_models_v1_mean.pkl",
    "experiments/evaluation_results_tuned_single_models_v2_weighted_CV.pkl",
    "experiments/evaluation_results_tuned_ensembles_test.pkl",
    "experiments/evaluation_results_tuned_LGBM_v3.pkl",
]

df_leaderboard_final = eval.generate_leaderboard_data(results_files=results_files_to_load)
eval.display_leaderboard(df_leaderboard_final, title="Tuned Ensemble Leaderboard")


#### **Check test set, because why not...**

In [ ]:
# --- 1. Imports ---
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from rich.console import Console
from rich.table import Table
from tqdm.notebook import tqdm

# Make sure all our custom classes and helper functions are available
from utils.training import StackingEnsemble, _calculate_mape
from utils.evaluations import _get_model_instance
from sklearn.linear_model import RidgeCV, ElasticNetCV, LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# --- 2. Configuration ---

# Paths to the BEST versions of our tuned parameters
final_params_paths = {
    "LGBMRegressor": "experiments/tuned_params_LGBMRegressor_v5.pkl",
    "CatBoostRegressor": "experiments/tuned_params_CatBoostRegressor_v2.pkl",
    "Stack_ElasticNet_LGBM_Ridge": "experiments/tuned_params_final_Stack_ElasticNet_LGBM_Ridge_v2.pkl",
}

# Load all parameter dictionaries into memory
loaded_params = {name: joblib.load(path) for name, path in final_params_paths.items()}

# --- 3. Final Evaluation Loop ---

console = Console()
final_test_results = {}

# A list of models to train and evaluate
models_to_test = [
    "RidgeCV",
    "LinearRegression",
    "Tuned_LGBM",
    "Tuned_CatBoost",
    "Avg_Ridge_TunedLGBM",
    "Stack_ElasticNet_LGBM_Ridge_Tuned",
]

for model_name in tqdm(models_to_test, desc="Final Test Set Evaluation"):
    horizon_predictions = {}

    for h in tqdm(config.HORIZONS, desc=f"Training {model_name}", leave=False):
        horizon_name = f"t+{h}"
        y_train_horizon = y_train[f"target_temp_{horizon_name}"]

        # --- Model-specific training on 100% of the training data ---

        if model_name == "RidgeCV":
            model = _get_model_instance(RidgeCV).fit(X_train_linear, y_train_horizon)
            y_pred_test = model.predict(X_test_linear)

        elif model_name == "LinearRegression":
            model = _get_model_instance(LinearRegression).fit(X_train_linear, y_train_horizon)
            y_pred_test = model.predict(X_test_linear)

        elif model_name == "Tuned_LGBM":
            params = loaded_params["LGBMRegressor"][horizon_name]
            model = _get_model_instance(LGBMRegressor, params=params).fit(X_train_tree, y_train_horizon)
            y_pred_test = model.predict(X_test_tree)

        elif model_name == "Tuned_CatBoost":
            params = loaded_params["CatBoostRegressor"][horizon_name]
            model = _get_model_instance(CatBoostRegressor, params=params).fit(X_train_tree, y_train_horizon)
            y_pred_test = model.predict(X_test_tree)

        elif model_name == "Avg_Ridge_TunedLGBM":
            # Train base models separately
            linear_base = _get_model_instance(RidgeCV).fit(X_train_linear, y_train_horizon)
            params_tree = loaded_params["LGBMRegressor"][horizon_name]
            tree_base = _get_model_instance(LGBMRegressor, params=params_tree).fit(X_train_tree, y_train_horizon)

            # Predict and average
            pred_linear = linear_base.predict(X_test_linear)
            pred_tree = tree_base.predict(X_test_tree)
            y_pred_test = (pred_linear + pred_tree) / 2.0

        elif model_name == "Stack_ElasticNet_LGBM_Ridge_Tuned":
            params_tree = loaded_params["Stack_ElasticNet_LGBM_Ridge"][horizon_name]
            tree_model_component = _get_model_instance(LGBMRegressor, params=params_tree)

            model = StackingEnsemble(
                base_model_linear=_get_model_instance(ElasticNetCV),
                base_model_tree=tree_model_component,
                meta_learner=_get_model_instance(RidgeCV),
            )
            model.fit({"linear": X_train_linear, "tree": X_train_tree}, y_train_horizon)
            y_pred_test = model.predict({"linear": X_test_linear, "tree": X_test_tree})

        horizon_predictions[horizon_name] = y_pred_test

    final_test_results[model_name] = horizon_predictions

# --- 4. Calculate and Display Metrics ---

results_summary = []
for model_name, horizon_preds in final_test_results.items():
    avg_metrics = {"r2": [], "rmse": [], "mae": [], "mape": []}
    for horizon_name, y_pred in horizon_preds.items():
        y_true = y_test[f"target_temp_{horizon_name}"]
        avg_metrics["r2"].append(r2_score(y_true, y_pred))
        avg_metrics["rmse"].append(np.sqrt(mean_squared_error(y_true, y_pred)))
        avg_metrics["mae"].append(mean_absolute_error(y_true, y_pred))
        avg_metrics["mape"].append(_calculate_mape(y_true.values, y_pred))

    results_summary.append({
        "Model": model_name,
        "Avg R²": np.mean(avg_metrics["r2"]),
        "Avg RMSE": np.mean(avg_metrics["rmse"]),
        "Avg MAE": np.mean(avg_metrics["mae"]),
        "Avg MAPE (%)": np.mean(avg_metrics["mape"]),
    })

df_summary = pd.DataFrame(results_summary).sort_values("Avg R²", ascending=False)

# Display using a simple Rich table
summary_table = Table(
    title="[bold]Final Model Performance on Held-Out Test Set[/bold]", show_header=True, header_style="bold blue"
)
summary_table.add_column("Model", style="cyan")
summary_table.add_column("Avg R² ↑", style="green")
summary_table.add_column("Avg RMSE ↓", style="yellow")
summary_table.add_column("Avg MAE ↓", style="yellow")
summary_table.add_column("Avg MAPE (%) ↓", style="yellow")

for _, row in df_summary.iterrows():
    summary_table.add_row(
        row["Model"],
        f"{row['Avg R²']:.4f}",
        f"{row['Avg RMSE']:.4f}",
        f"{row['Avg MAE']:.4f}",
        f"{row['Avg MAPE (%)']:.2f}%",
    )

console.print(summary_table)


<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"><span style="font-style: italic">                       </span><span style="font-weight: bold; font-style: italic">Final Model Performance on Held-Out Test Set</span><span style="font-style: italic">                       </span>
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃<span style="color: #000080; text-decoration-color: #000080; font-weight: bold"> Model                             </span>┃<span style="color: #000080; text-decoration-color: #000080; font-weight: bold"> Avg R² ↑ </span>┃<span style="color: #000080; text-decoration-color: #000080; font-weight: bold"> Avg RMSE ↓ </span>┃<span style="color: #000080; text-decoration-color: #000080; font-weight: bold"> Avg MAE ↓ </span>┃<span style="color: #000080; text-decoration-color: #000080; font-weight: bold"> Avg MAPE (%) ↓ </span>┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│<span style="color: #008080; text-decoration-color: #008080"> Avg_Ridge_TunedLGBM               </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.6198   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9660     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7695    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.69%          </span>│
│<span style="color: #008080; text-decoration-color: #008080"> Stack_ElasticNet_LGBM_Ridge_Tuned </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.6175   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9687     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7715    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.69%          </span>│
│<span style="color: #008080; text-decoration-color: #008080"> LinearRegression                  </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.6137   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9740     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7746    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.71%          </span>│
│<span style="color: #008080; text-decoration-color: #008080"> RidgeCV                           </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.6137   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9740     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7746    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.71%          </span>│
│<span style="color: #008080; text-decoration-color: #008080"> Tuned_LGBM                        </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.6031   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9871     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7888    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.74%          </span>│
│<span style="color: #008080; text-decoration-color: #008080"> Tuned_CatBoost                    </span>│<span style="color: #008000; text-decoration-color: #008000"> 0.5958   </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.9955     </span>│<span style="color: #808000; text-decoration-color: #808000"> 0.7974    </span>│<span style="color: #808000; text-decoration-color: #808000"> 2.77%          </span>│
└───────────────────────────────────┴──────────┴────────────┴───────────┴────────────────┘
</pre>

In [ ]:
# sandbox_notebook.ipynb - Final Test Set Performance Comparison

# --- 1. Imports ---
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from rich.console import Console
from rich.table import Table
from tqdm.notebook import tqdm
import os

# Make sure all our custom classes and helper functions are available
# Ensure this path is correct relative to your sandbox notebook
from utils.training import StackingEnsemble, _calculate_mape
from utils.evaluations import _get_model_instance
from sklearn.linear_model import RidgeCV, ElasticNetCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# --- 2. Configuration for this Experiment ---

# This dictionary is your control panel for this specific test.
# It compares models tuned with a standard mean objective vs. our weighted objective.
models_to_evaluate = {
    # CatBoost Single Models
    "CatBoost_v1_mean": {
        "type": "single_tuned",
        "tree_model_class": CatBoostRegressor,
        "params_path": "experiments/tuned_params_CatBoostRegressor_v1.pkl",
    },
    "CatBoost_v2_weighted": {
        "type": "single_tuned",
        "tree_model_class": CatBoostRegressor,
        "params_path": "experiments/tuned_params_CatBoostRegressor_v2.pkl",
    },
    # LGBM Single Models
    "LGBM_v1_mean": {
        "type": "single_tuned",
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v1.pkl",
    },
    "LGBM_v2_weighted": {
        "type": "single_tuned",
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    },
    # Simple Averaging Ensembles
    "Avg_Ridge_LGBM_v1_mean": {
        "type": "simple_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v1.pkl",
    },
    "Avg_Ridge_LGBM_v2_weighted": {
        "type": "simple_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    },
    # Single models
    "RidgeCV": {
        "type": "oob",
        "model_class": RidgeCV,
        "feature_set": "linear",
    },
    "ElasticNetCV": {
        "type": "oob",
        "model_class": ElasticNetCV,
        "feature_set": "linear",
    },
}

# --- 3. Final Evaluation Loop ---

console = Console()
final_test_results = {}

for model_name, model_config in tqdm(models_to_evaluate.items(), desc="Final Test Set Evaluation"):
    horizon_predictions = {}
    model_type = model_config.get("type")

    if model_type != "oob":
        # Load parameters once per model
        params_path = model_config.get("params_path")
        if not params_path or not os.path.exists(params_path):
            print(f"Skipping {model_name}: Param file not found at {params_path}")
            continue
        loaded_params = joblib.load(params_path)

    for h in tqdm(config.HORIZONS, desc=f"Training {model_name}", leave=False):
        horizon_name = f"t+{h}"
        y_train_horizon = y_train[f"target_temp_{horizon_name}"]

        params_for_horizon = loaded_params[horizon_name]

        # --- Model-specific training on 100% of the training data ---

        if model_type == "single_tuned":
            model = _get_model_instance(model_config["tree_model_class"], params=params_for_horizon)
            model.fit(X_train_tree, y_train_horizon)
            y_pred_test = model.predict(X_test_tree)

        elif model_type == "simple_averaging_tuned":
            linear_base = _get_model_instance(model_config["linear_model_class"]).fit(X_train_linear, y_train_horizon)
            tree_base = _get_model_instance(model_config["tree_model_class"], params=params_for_horizon).fit(
                X_train_tree, y_train_horizon
            )

            pred_linear = linear_base.predict(X_test_linear)
            pred_tree = tree_base.predict(X_test_tree)
            y_pred_test = (pred_linear + pred_tree) / 2.0

        elif model_name == "RidgeCV":
            model = _get_model_instance(RidgeCV).fit(X_train_linear, y_train_horizon)
            y_pred_test = model.predict(X_test_linear)

        elif model_name == "ElasticNetCV":
            model = _get_model_instance(ElasticNetCV).fit(X_train_linear, y_train_horizon)
            y_pred_test = model.predict(X_test_linear)

        horizon_predictions[horizon_name] = y_pred_test

    final_test_results[model_name] = horizon_predictions

# --- 4. Calculate and Display Metrics (Enhanced Table) ---

results_summary = []
for model_name, horizon_preds in final_test_results.items():
    per_horizon_r2 = {}
    avg_metrics = {"r2": [], "rmse": [], "mae": [], "mape": []}

    for h in config.HORIZONS:
        horizon_name = f"t+{h}"
        y_pred = horizon_preds[horizon_name]
        y_true = y_test[f"target_temp_{horizon_name}"]

        r2 = r2_score(y_true, y_pred)
        per_horizon_r2[f"R² (t+{h})"] = r2

        avg_metrics["r2"].append(r2)
        avg_metrics["rmse"].append(np.sqrt(mean_squared_error(y_true, y_pred)))
        avg_metrics["mae"].append(mean_absolute_error(y_true, y_pred))
        avg_metrics["mape"].append(_calculate_mape(y_true.values, y_pred))

    summary_row = {
        "Model": model_name,
        "Avg R²": np.mean(avg_metrics["r2"]),
        "Avg RMSE": np.mean(avg_metrics["rmse"]),
        "Avg MAE": np.mean(avg_metrics["mae"]),
        "Avg MAPE (%)": np.mean(avg_metrics["mape"]),
        **per_horizon_r2,  # Unpack the per-horizon R² scores into the dictionary
    }
    results_summary.append(summary_row)

df_summary = pd.DataFrame(results_summary).sort_values("Avg R²", ascending=False)

# Display using a Rich table with per-horizon metrics
summary_table = Table(
    title="[bold]Test Set Performance: Weighted CV vs. Mean CV Tuning Objective[/bold]",
    show_header=True,
    header_style="bold blue",
    show_lines=True,
)
summary_table.add_column("Model", style="cyan", width=30)
summary_table.add_column("Avg R² ↑", style="bold green", justify="center")

# Add per-horizon R² columns
for h in config.HORIZONS:
    summary_table.add_column(f"R² (t+{h})", justify="center")

# Add other average metrics
summary_table.add_column("Avg RMSE ↓", style="yellow", justify="center")
summary_table.add_column("Avg MAE ↓", style="yellow", justify="center")

for _, row in df_summary.iterrows():
    table_row_data = [
        row["Model"],
        f"{row['Avg R²']:.4f}",
        f"{row['R² (t+1)']:.4f}",
        f"{row['R² (t+2)']:.4f}",
        f"{row['R² (t+3)']:.4f}",
        f"{row['R² (t+4)']:.4f}",
        f"{row['R² (t+5)']:.4f}",
        f"{row['Avg RMSE']:.4f}",
        f"{row['Avg MAE']:.4f}",
    ]
    summary_table.add_row(*table_row_data)

console.print(summary_table)


#### **FLAML test**

In [ ]:
# sandbox_notebook.ipynb - AutoML Comparison: FLAML

# --- 1. Imports ---
from flaml import AutoML
import pandas as pd
import numpy as np
from rich.console import Console
from rich.table import Table
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import time
from utils.evaluations import _calculate_mape

# --- 2. Reusable AutoML Evaluation Function (for both FLAML & AutoGluon) ---


def evaluate_and_display_automl(
    predictor_dict: dict,
    model_family_name: str,
    X_test_linear: pd.DataFrame,
    X_test_tree: pd.DataFrame,
    y_test: pd.DataFrame,
    console: Console,
):
    """
    Takes a dictionary of trained per-horizon AutoML predictors, evaluates them on the
    test set, and displays the results in our standard rich table format.
    """
    final_test_results = {}

    # --- A. Generate Predictions ---
    for horizon_name, predictor in predictor_dict.items():
        # AutoML libraries typically use the tree-based feature set
        y_pred_test = predictor.predict(X_test_tree)
        final_test_results[horizon_name] = y_pred_test

    # --- B. Calculate Metrics ---
    results_summary = []
    per_horizon_r2 = {}
    avg_metrics = {"r2": [], "rmse": [], "mae": [], "mape": []}

    for h in config.HORIZONS:
        horizon_name = f"t+{h}"
        y_pred = final_test_results[horizon_name]
        y_true = y_test[f"target_temp_{horizon_name}"]

        r2 = r2_score(y_true, y_pred)
        per_horizon_r2[f"R² ({horizon_name})"] = r2
        avg_metrics["r2"].append(r2)
        avg_metrics["rmse"].append(np.sqrt(mean_squared_error(y_true, y_pred)))
        avg_metrics["mae"].append(mean_absolute_error(y_true, y_pred))
        avg_metrics["mape"].append(_calculate_mape(y_true.values, y_pred))

    summary_row = {
        "Model": model_family_name,
        "Avg R²": np.mean(avg_metrics["r2"]),
        "Avg RMSE": np.mean(avg_metrics["rmse"]),
        "Avg MAE": np.mean(avg_metrics["mae"]),
        "Avg MAPE (%)": np.mean(avg_metrics["mape"]),
        **per_horizon_r2,
    }
    results_summary.append(summary_row)
    df_summary = pd.DataFrame(results_summary)

    # --- C. Display Table ---
    summary_table = Table(
        title=f"[bold]Test Set Performance: {model_family_name}[/bold]",
        show_header=True,
        header_style="bold blue",
        show_lines=True,
    )
    summary_table.add_column("Model", style="cyan", width=30)
    summary_table.add_column("Avg R² ↑", style="bold green", justify="center")
    for h in config.HORIZONS:
        summary_table.add_column(f"R² (t+{h})", justify="center")
    summary_table.add_column("Avg RMSE ↓", style="yellow", justify="center")
    summary_table.add_column("Avg MAE ↓", style="yellow", justify="center")

    for _, row in df_summary.iterrows():
        table_row_data = [
            row["Model"],
            f"{row['Avg R²']:.4f}",
            f"{row['R² (t+1)']:.4f}",
            f"{row['R² (t+2)']:.4f}",
            f"{row['R² (t+3)']:.4f}",
            f"{row['R² (t+4)']:.4f}",
            f"{row['R² (t+5)']:.4f}",
            f"{row['Avg RMSE']:.4f}",
            f"{row['Avg MAE']:.4f}",
        ]
        summary_table.add_row(*table_row_data)

    console.print(summary_table)

    return final_test_results  # Return predictions for potential further analysis


In [ ]:
# --- 3. FLAML Experiment ---

console = Console()
flaml_predictors = {}
TIME_BUDGET_PER_HORIZON = 600

print("\n" + "=" * 50)
print(f"  STARTING: FLAML Evaluation (Budget: {TIME_BUDGET_PER_HORIZON}s per horizon)")
print("=" * 50 + "\n")

total_start_time = time.time()

for h in tqdm(config.HORIZONS, desc="FLAML Horizon Evaluation"):
    horizon_name = f"t+{h}"
    y_train_horizon = y_train[f"target_temp_{horizon_name}"]

    print(f"\n--- Training FLAML for {horizon_name} ---")

    # Initialize AutoML for this horizon
    automl = AutoML()

    # Define settings
    automl_settings = {
        "time_budget": TIME_BUDGET_PER_HORIZON,
        "metric": "r2",
        "task": "regression",
        "log_file_name": f"experiments/flaml_{horizon_name}.log",
        "n_jobs": -1,
        "seed": config.GLOBAL_RANDOM_SEED,
        # "estimator_list": ["lgbm", "xgboost", "catboost", "rf", "extra_tree"],  # Focus on tree models
    }

    # Train FLAML on the full training data for this horizon
    automl.fit(X_train=X_train_tree, y_train=y_train_horizon, **automl_settings)

    # Store the trained predictor
    flaml_predictors[horizon_name] = automl

    print(f"--- Best model for {horizon_name}: {automl.best_estimator} ---")
    print(f"Best R2 score on internal validation: {automl.best_result['val_loss']:.4f}")

total_end_time = time.time()
print(f"\n--- FLAML Training Complete. Total time: {(total_end_time - total_start_time) / 60:.2f} minutes ---")



In [ ]:
# --- 4. Evaluate and Display FLAML's Test Set Performance ---
flaml_test_predictions = evaluate_and_display_automl(
    predictor_dict=flaml_predictors,
    model_family_name=f"FLAML (Budget: {TIME_BUDGET_PER_HORIZON}s/horizon)",
    X_test_linear=X_test_linear,  # Not used by FLAML but required by function signature
    X_test_tree=X_test_tree,
    y_test=y_test,
    console=console,
)


#### **Autogluon test**

In [ ]:
# sandbox_notebook.ipynb - AutoML Comparison: AutoGluon

# --- 1. Imports ---
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
from rich.console import Console
from rich.table import Table
from tqdm.notebook import tqdm
import time
import os

# --- 2. AutoGluon Experiment ---

console = Console()
autogluon_predictors = {}
TIME_LIMIT_PER_HORIZON = 1000

print("\n" + "=" * 50)
print(f"  STARTING: AutoGluon Evaluation (Budget: {TIME_LIMIT_PER_HORIZON}s per horizon)")
print("=" * 50 + "\n")

total_start_time = time.time()

# AutoGluon requires the target column to be part of the training DataFrame.
# We will create temporary DataFrames for each horizon.
train_data_ag = X_train_tree.copy()
for col in y_train.columns:
    train_data_ag[col] = y_train[col]


for h in tqdm(config.HORIZONS, desc="AutoGluon Horizon Evaluation"):
    horizon_name = f"t+{h}"
    target_col = f"target_temp_{horizon_name}"

    # Define a unique path for this predictor's artifacts
    predictor_path = f"experiments/autogluon/h{h}"
    os.makedirs(predictor_path, exist_ok=True)

    print(f"\n--- Training AutoGluon for {horizon_name} ---")

    # Initialize TabularPredictor for this horizon
    predictor = TabularPredictor(
        label=target_col,
        path=predictor_path,
        eval_metric="r2",
    )

    # Train the predictor on the full training data
    # We drop the other target columns to prevent data leakage
    predictor.fit(
        train_data=train_data_ag.drop(columns=[c for c in target_cols if c != target_col]),
        time_limit=TIME_LIMIT_PER_HORIZON,
        presets="best_quality",
        # Exclude models that are known to be slow or less effective for this data type
        excluded_model_types=["FASTAI", "KNN"],
    )

    # Store the trained predictor object
    autogluon_predictors[horizon_name] = predictor

    print(f"--- AutoGluon training for {horizon_name} complete. ---")
    # You can view the detailed leaderboard for this horizon's predictor:
    predictor.leaderboard(train_data_ag, silent=True)


total_end_time = time.time()
print(f"\n--- AutoGluon Training Complete. Total time: {(total_end_time - total_start_time) / 60:.2f} minutes ---")


In [ ]:
# --- 3. Evaluate and Display AutoGluon's Test Set Performance ---
autogluon_test_predictions = evaluate_and_display_automl(
    predictor_dict=autogluon_predictors,
    model_family_name=f"AutoGluon (Preset: 'best_quality', Budget: {TIME_LIMIT_PER_HORIZON}s/horizon)",
    X_test_linear=X_test_linear,  # Not used, but required by function signature
    X_test_tree=X_test_tree,
    y_test=y_test,
    console=console,
)


#### **Check extrapolation limit**

In [ ]:
# sandbox_notebook.ipynb - Diagnostic: Extrapolation Failure

# --- 1. Imports ---
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor
import utils.analysis_helpers as ah

# --- 2. Train Diagnostic Models ---
# We train on 100% of the training data for the t+1 horizon, as this has the strongest signal.
console.print("[bold]Training diagnostic models on full training set for t+1 horizon...[/bold]")

# Train RidgeCV
model_ridge_diag = RidgeCV(cv=TimeSeriesSplit(n_splits=5)).fit(X_train_linear, y_train["target_temp_t+1"])

# Train OOB LGBM
model_lgbm_diag = LGBMRegressor(random_state=config.GLOBAL_RANDOM_SEED, n_jobs=-1, verbosity=-1).fit(
    X_train_tree, y_train["target_temp_t+1"]
)

# --- 3. Generate Predictions on the Test Set ---
y_pred_ridge_test = pd.Series(model_ridge_diag.predict(X_test_linear), index=X_test_linear.index)
y_pred_lgbm_test = pd.Series(model_lgbm_diag.predict(X_test_tree), index=X_test_tree.index)

# --- 4. Display the Quantitative Summary ---
ah.display_extrapolation_summary(
    y_train=y_train["target_temp_t+1"],
    y_test=y_test["target_temp_t+1"],
    y_pred_linear=y_pred_ridge_test,
    y_pred_tree=y_pred_lgbm_test,
)


In [ ]:
fig_diag = ah.plot_extrapolation_diagnostics(
    y_train=y_train["target_temp_t+1"],
    y_test=y_test["target_temp_t+1"],
    y_pred_linear=y_pred_ridge_test,
    y_pred_tree=y_pred_lgbm_test,
)

fig_diag.show()


#### **Horizon-weighted average ensembles training**

In [ ]:
# sandbox_notebook.ipynb - Final Refinement: Horizon-Weighted Averaging

# --- 1. Configuration ---
# We will test our best linear and best tuned tree models
models_to_evaluate_weighted = {
    "WeightedAvg_Ridge_TunedLGBM": {
        "type": "weighted_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "./experiments/tuned_params_LGBMRegressor_v3.pkl",
    },
    # "WeightedAvg_Ridge_TunedCatBoost": {
    #     "type": "weighted_averaging_tuned",
    #     "linear_model_class": RidgeCV,
    #     "tree_model_class": CatBoostRegressor,
    #     "params_path": "experiments/tuned_params_final_CatBoostRegressor.pkl",
    # },
}

# --- 2. Run the Evaluation ---
print("\n--- Searching for Optimal Ensemble Weights ---")
weighted_avg_results = eval.evaluate_models(
    models_to_evaluate=models_to_evaluate_weighted,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(weighted_avg_results, "experiments/evaluation_results_tuned_ensemble_weighted_avg.pkl")
print("Weighted average ensemble results saved to 'experiments/evaluation_results_tuned_ensemble_weighted_avg.pkl'")

# --- 3. Analyze the Optimal Weights Found ---
# This part can be run in the presentation notebook later, but it's good to check here.
console = Console()
weights_table = Table(title="[bold]Optimal Blending Weight for Linear Model (RidgeCV)[/bold]")
weights_table.add_column("Horizon", style="cyan")
weights_table.add_column("Avg. Optimal Weight", justify="center", style="green")

for model_name, model_results in weighted_avg_results.items():
    for h in config.HORIZONS:
        horizon_name = f"t+{h}"
        avg_weight = np.mean(model_results[horizon_name]["scores"]["optimal_weights"])
        weights_table.add_row(f"({model_name}) {horizon_name}", f"{avg_weight:.2f}")

console.print(weights_table)


#### **Top 2 models snapshot**

In [ ]:
DIAGNOSTIC_ARTIFACT_PATH = "experiments/chosen_models_diagnostic_data.pkl"

# Define our two champion models for final diagnostics
final_models_to_diagnose = {
    "WeightedAvg_Ridge_TunedLGBM": {
        "type": "weighted_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
        "horizon_weights": {1: 0.40, 2: 0.39, 3: 0.53, 4: 0.50, 5: 0.58},
    },
    "RidgeCV": {
        "type": "oob",
        "model_class": RidgeCV,
        "feature_set": "linear",
    },
}


In [ ]:
# --- 2. Generate or Load the Data ---
# This function will either run the full CV process or load from the cache.
diagnostic_data = diag.generate_diagnostic_data(
    models_to_run=final_models_to_diagnose,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons_to_analyze=config.HORIZONS,
    artifact_path=DIAGNOSTIC_ARTIFACT_PATH,
)


#### **Generate feature permuation importance**

In [ ]:
SHAP_VS_PERM_ARTIFACT_PATH = "assets/saved_results/shap_vs_perm_data.pkl"

champion_model_config = {
    "WeightedAvg_Ridge_TunedLGBM": {
        "type": "weighted_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
        "horizon_weights": {1: 0.40, 2: 0.39, 3: 0.53, 4: 0.50, 5: 0.58},
    }
}
# Extract the single config dictionary needed by the function
champion_config_for_function = list(champion_model_config.values())[0]


# --- 2. Generate or Load the Data ---
# This function will either run the full CV and SHAP process or load from the cache.
shap_vs_perm_data = diag.generate_shap_vs_perm_data(
    champion_model_config=champion_config_for_function,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons_to_analyze=[1, 2, 3, 4, 5],
    artifact_path=SHAP_VS_PERM_ARTIFACT_PATH,
    n_repeats=20,
)

print("\n--- SHAP vs. Permutation Data Generation Complete ---")


#### **Test enhancements and all**

In [ ]:
# sandbox_notebook.ipynb - Final Enhancement Experiments

# --- 1. Base Champion Config ---
base_champion_config = {
    "type": "weighted_averaging_tuned",
    "linear_model_class": RidgeCV,
    "tree_model_class": LGBMRegressor,
    "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    "horizon_weights": {1: 0.40, 2: 0.39, 3: 0.53, 4: 0.50, 5: 0.58},
}

# --- 2. Enhancement Experiment Configurations ---
enhancement_configs_to_run = {
    "Ensemble + Isotonic Calibration": {
        "type": "calibration",
    },
    "Ensemble + Residual Stacking (Tuned LGBM)": {
        "type": "residual_stacking",
        "error_model_class": LGBMRegressor,
        "error_model_feature_set": "tree",
        # CRITICAL: Use the best tuned params for the error model
        "error_model_params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    },
    "Ensemble + Residual Stacking (Ridge)": {
        "type": "residual_stacking",
        "error_model_class": RidgeCV,
        "error_model_feature_set": "linear",
    },
    # NEW: The combined "kitchen sink" approach
    "Ensemble + Residual Stacking (Tuned LGBM) + Calibration": {
        "type": "residual_stacking_calibration",
        "error_model_class": LGBMRegressor,
        "error_model_feature_set": "tree",
        "error_model_params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
    },
}

# --- 3. Run the Evaluation ---
print("\n--- Running Final Evaluation of Ensemble Model Enhancements ---")
enhancement_results = eval.evaluate_enhancements(
    base_model_config=base_champion_config,
    enhancement_configs=enhancement_configs_to_run,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    cv_splitter=tscv_splitter,
    horizons=config.HORIZONS,
)
joblib.dump(enhancement_results, "experiments/evaluation_results_ensemble_enhancements.pkl")
print("Final enhancement evaluation results saved to 'experiments/evaluation_results_ensemble_enhancements.pkl'")


#### **Test set, FINALLY**

In [27]:
final_models_to_train = {
    "WeightedAvg_Ridge_TunedLGBM": {
        "type": "weighted_averaging_tuned",
        "linear_model_class": RidgeCV,
        "tree_model_class": LGBMRegressor,
        "params_path": "experiments/tuned_params_LGBMRegressor_v3.pkl",
        "horizon_weights": {1: 0.40, 2: 0.39, 3: 0.53, 4: 0.50, 5: 0.58},
    },
    "RidgeCV": {
        "type": "oob",
        "model_class": RidgeCV,
        "feature_set": "linear",
    },
}

# --- 2. Run Final Training ---
train.train_and_save_final_models(
    models_to_train=final_models_to_train,
    X_train_linear=X_train_linear,
    X_train_tree=X_train_tree,
    y_train=y_train,
    horizons=config.HORIZONS,
    save_dir="assets/models",
)


Training final models on 100% of training data...

Training Final Models:   0%|          | 0/2 [00:00<?, ?it/s]

All final models trained and saved to 'assets/models'.

#### **Lagged rolling average benchmarks...**

In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from rich.console import Console
from rich.table import Table


def test_lagged_average_baseline(
    y_true_all_horizons: pd.DataFrame, full_temp_series: pd.Series, horizons: list, windows: list
):
    """
    Calculates the R² score for a time-lagged rolling average baseline
    on a test set for multiple horizons and window sizes, ensuring no data leakage.

    Args:
        y_true_all_horizons (pd.DataFrame): DataFrame containing the true target values
                                            for all horizons on the test set.
        full_temp_series (pd.Series): The complete, original 'temp' series spanning
                                      both training and test periods.
        horizons (list): A list of integer horizons to test (e.g., [1, 5]).
        windows (list): A list of integer window sizes for the rolling average (e.g., [7, 30]).
    """
    console = Console()
    results_table = Table(
        title="[bold]Time-Lagged Average Baseline Performance (R² on Test Set)[/bold]",
        show_header=True,
        header_style="bold magenta",
    )
    results_table.add_column("Horizon", style="cyan")
    for w in windows:
        results_table.add_column(f"Window = {w} days", justify="center")

    # The test set starts on the first index of our y_true DataFrame
    test_start_date = y_true_all_horizons.index.min()

    # To prevent any leakage, we only use data *before* the test set begins
    # to calculate the rolling means that will be used for predictions.
    temp_series_for_rolling = full_temp_series.loc[: test_start_date - pd.Timedelta(days=1)]

    for h in horizons:
        horizon_name = f"t+{h}"
        y_true_horizon = y_true_all_horizons[f"target_temp_{horizon_name}"]
        row_scores = [horizon_name]

        for w in windows:
            # --- CRITICAL LEAKAGE PREVENTION ---
            # 1. Calculate the rolling mean on historical data ONLY.
            rolling_mean = temp_series_for_rolling.rolling(window=w).mean()

            # 2. To predict for a day `d` in the test set, we need the rolling average
            #    calculated up to day `d-h`. We achieve this by shifting the
            #    historical rolling mean series forward by `h` days.
            #    This correctly simulates that for a t+5 forecast, we only have data
            #    available up to day t.

            # Create a shifted index to align predictions with the test set dates
            shifted_index = rolling_mean.index + pd.Timedelta(days=h)

            # Create the prediction series with the correctly shifted index
            y_pred_series = pd.Series(rolling_mean.values, index=shifted_index)

            # 3. Align the predictions with the true values for this horizon
            y_pred_aligned = y_pred_series.reindex(y_true_horizon.index).ffill().bfill()

            # --- Calculate Score ---
            score = r2_score(y_true_horizon, y_pred_aligned)
            row_scores.append(f"{score:.4f}")

        results_table.add_row(*row_scores)

    console.print(results_table)


In [29]:
full_temp_series_from_ready = df_model_ready["temp"]

horizons_to_test = [
    1,
    2,
    3,
    4,
    5,
]
windows_to_test = [
    1,
    3,
    7,
    14,
    28,
    30,
    365,
]

# Run the test
test_lagged_average_baseline(
    y_true_all_horizons=y_test,
    full_temp_series=full_temp_series_from_ready,
    horizons=horizons_to_test,
    windows=windows_to_test,
)


                             Time-Lagged Average Baseline Performance (R² on Test Set)                             
┏━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃         ┃  Window = 1  ┃  Window = 3  ┃  Window = 7  ┃ Window = 14  ┃ Window = 28  ┃ Window = 30 ┃ Window = 365 ┃
┃ Horizon ┃     days     ┃     days     ┃     days     ┃     days     ┃     days     ┃    days     ┃     days     ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ t+1     │   -0.0002    │   -0.0147    │   -0.2848    │   -0.0703    │   -0.0098    │   -0.0091   │   -0.0022    │
│ t+2     │   -0.0002    │   -0.0152    │   -0.2869    │   -0.0716    │   -0.0103    │   -0.0097   │   -0.0020    │
│ t+3     │    0.0002    │   -0.0152    │   -0.2891    │   -0.0729    │   -0.0109    │   -0.0103   │   -0.0017    │
│ t+4     │    0.0008    │   -0.0153    │   -0.2920    │   -0.0745    │   -0.0115    │   -0.0110   │   -0.0015    │
│ t+5     │    0.0022    │   -0.0148    │   -0.2950    │   -0.0764    │   -0.0125    │   -0.0119   │   -0.0013    │
└─────────┴──────────────┴──────────────┴──────────────┴──────────────┴──────────────┴─────────────┴──────────────┘